In [3]:
project = 'derivative_project05'
import antropy as ant
from antropy import lziv_complexity
import matplotlib
#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show
import pathlib
import mne
import os
import os.path as op
from mne import viz
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import zipfile
import sys
import pandas as pd
import numpy as np
from IPython.utils import io
import neurokit2 as nk
import scipy
from scipy import stats
import scipy.stats
import seaborn as sns
import pickle
import itertools
from whatever import *
import time

In [4]:
# Subjects
sub_final = sorted(os.listdir('./data'))
print('    > subjects:', len (sub_final))

# Files
fol_names = []
for i in sub_final:
    subfols = os.listdir(f'./data/{i}')
    for s in subfols:
        fol_names.append(s)
print('    > files:',len(fol_names))

## OLD used to be: 
    #> subjects: 57
    #> files: 132
## NEW not its:
    #> subjects: 70
    #> files: 169


    > subjects: 70
    > files: 169


In [5]:
#13 New Participants Adding to data 
New=sorted(['sub-CCTC07NETICU','sub-NETICU31','sub-NETICU33','sub-NETICU38','sub-NETICU40','sub-NETICU41','sub-NETICU46','sub-CCTC07','sub-MSICU32','sub-MSICU38','sub-MSICU42','sub-MSICU43','sub-MSICU47'])
print(len(New))
sub_final = New.copy()

# Files
fol_names = []
for i in sub_final:
    subfols = os.listdir(f'./data/{i}')
    for s in subfols:
        fol_names.append(s)
print('    > files:',len(fol_names))

13
    > files: 37


In [6]:
sub_final

['sub-CCTC07',
 'sub-CCTC07NETICU',
 'sub-MSICU32',
 'sub-MSICU38',
 'sub-MSICU42',
 'sub-MSICU43',
 'sub-MSICU47',
 'sub-NETICU31',
 'sub-NETICU33',
 'sub-NETICU38',
 'sub-NETICU40',
 'sub-NETICU41',
 'sub-NETICU46']

In [7]:
fol_names

['sub-CCTC07_task-rest_eeg',
 'sub-CCTC07_task-taken_eeg',
 'sub-CCTC07NETICU_task-sedon1rest_eeg',
 'sub-CCTC07NETICU_task-sedoffrest_eeg',
 'sub-CCTC07NETICU_task-sedofftaken_eeg',
 'sub-CCTC07NETICU_task-sedon1taken_eeg',
 'sub-MSICU32_task-rest_eeg',
 'sub-MSICU32_task-taken_eeg',
 'sub-MSICU38_task-taken_eeg',
 'sub-MSICU38_task-rest_eeg',
 'sub-MSICU42_task-rest_eeg',
 'sub-MSICU42_task-taken_eeg',
 'sub-MSICU43_task-rest_eeg',
 'sub-MSICU43_task-taken_eeg',
 'sub-MSICU47_task-rest_eeg',
 'sub-MSICU47_task-taken_eeg',
 'sub-NETICU31_task-sedofftaken_eeg',
 'sub-NETICU31_task-sedon2rest_eeg',
 'sub-NETICU31_task-sedon1taken_eeg',
 'sub-NETICU31_task-sedon1rest_eeg',
 'sub-NETICU33_task-sedoffrest_eeg',
 'sub-NETICU33_task-sedon1taken_eeg',
 'sub-NETICU33_task-sedon1rest_eeg',
 'sub-NETICU38_task-sedon1rest_eeg',
 'sub-NETICU38_task-sedon1taken_eeg',
 'sub-NETICU40_task-sedoffrest_eeg',
 'sub-NETICU40_task-sedon1taken_eeg',
 'sub-NETICU40_task-sedon1rest_eeg',
 'sub-NETICU40_task-s

In [8]:
HC= []
for f in fol_names:
    if 'HC' in f:
        HC.append(f)
len(HC)

0

# Crop, Downsample, & Save New Files

In [10]:
matplotlib.use('Qt5Agg')

#####################
### Cropping data ###
#####################

var_names = []
print ("####################################################################")
print ("### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###")
print ("### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###")
print ("####################################################################")
#for f in ['sub-CCTC03_task-taken_eeg','sub-CCTC03_task-rest02_eeg']:
for f in fol_names:
# for f in ['sub-HC05_task-Taken_eeg']:
    print ('\n______________________',f,'______________________')
    ID = ((f[4:]).split("_task"))[0] # f[4:] removes 'sub-' from f, Then the .split() will split it every "_task", Then [0] picks the ID "CCTC03"
    source_path = f'./data/sub-{ID}/{f}'
    fname = (sorted(os.listdir(source_path)))[0]
    data_path = f'{source_path}/{fname}'
    # Loading data
    with io.capture_output() as captured:
        raw = mne.io.read_raw_egi(data_path, preload=True)
    data = raw.copy()
    raw_sfreq = raw.info['sfreq']

    #Timing############################################################################################
    meas_data = data.info['meas_date']
    print(f'\n    - {fname}',
          f'\n    - {meas_data}',
          f'\n    - {data}',
          f'\n    - time(s): {(data.last_samp+1)/raw_sfreq}',
          f'\n    - first_samp-last_samp: {data.first_samp} - {data.last_samp}')
    seconds = (data.last_samp+1)/raw_sfreq
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    print("    - Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))
    #EVENTS############################################################################################
    triggers = []
    for i in data.ch_names:
        if not 'E' in i:
            if not 'sync' in i:
                if not 'STI' in i:
                    if not 'V' in i:
                        print (f"    - Trigger <  ch. {i}>")
                        triggers.append(i)
    with io.capture_output() as captured:
        events = mne.find_events(data, stim_channel='STI 014', initial_event=True)
    for i in range(events.shape[0]):
        vars()['event'+str(i+1)] = events[i,0]
        print (f'{i+1} - {events[i,0]} / {raw_sfreq} = {(events[i,0])/raw_sfreq}')

    #crop####################################################################################################
    for i in range(events.shape[0]):
        name = input(f'Name event{i+1} for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name event{i+1} for {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < event{i+1} for {f}\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            if '_smin' in name:
                name = (name[:-5])
                smin = input(f'> smin to replace event{i+1} start time (type "zoom" to plot a closer look):')
                if smin == 'zoom':
                    with io.capture_output() as captured:
                        data.plot(duration=10,
                                  n_channels=10,
                                  scalings=1000e-6, #20e-6`` (20µV)
                                  #show_options=True,
                                  #group_by='position',
                                  butterfly=False,
                                  title=f,
                                  clipping=None,
                                  block=True)
                    time.sleep(2)
                    smin = int(input(f'> smin to replace event{i+1} start time:'))
                else:
                    smin = int(smin)
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')
            else:
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=vars()['event'+str(i+1)], smax=vars()['event'+str(i+1)]+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')# 15x(_______________)
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
    #if no triggers
    if len(events) == 0 and len(triggers) == 0:
        name = input(f'Name file for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name for file {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < for {f} (no events)\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            missingtrig_smin = int(input(f'> smin value (samplingrate={raw_sfreq}) for {f}:'))
            vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=missingtrig_smin, smax=missingtrig_smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{name}_______________{f}')
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
var_names



####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

______________________ sub-CCTC07_task-rest_eeg ______________________

    - Session 20240325 1414.mff 
    - 2024-03-25 19:25:43.762000+00:00 
    - <RawMff | signal1.bin, 133 x 310630 (310.6 s), ~315.4 MB, data loaded> 
    - time(s): 310.63 
    - first_samp-last_samp: 0 - 310629
    - Duration = 0hr : 05min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5051 / 1000.0 = 5.051
2 - 305139 / 1000.0 = 305.139
Name event1 for sub-CCTC07_task-rest_eeg:_RS
Name event2 for sub-CCTC07_task-rest_eeg:no

______________________ sub-CCTC07_task-taken_eeg ______________________

    - Session 20240325 1431.mff 
    - 2024-03-25 19:31:35.663000+00:00 
    - <RawMff | signal1.bin, 135 x 643354 (643.4 s), ~6


    - Session 20240417 1005.mff 
    - 2024-04-17 15:13:20.578000+00:00 
    - <RawMff | signal1.bin, 133 x 360475 (360.5 s), ~366.0 MB, data loaded> 
    - time(s): 360.475 
    - first_samp-last_samp: 0 - 360474
    - Duration = 0hr : 06min : 00sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5070 / 1000.0 = 5.07
2 - 355144 / 1000.0 = 355.144
Name event1 for sub-MSICU43_task-rest_eeg:_RS
Name event2 for sub-MSICU43_task-rest_eeg:no

______________________ sub-MSICU43_task-taken_eeg ______________________

    - Session 20240417 1019.mff 
    - 2024-04-17 15:21:21.902000+00:00 
    - <RawMff | signal1.bin, 135 x 643505 (643.5 s), ~663.0 MB, data loaded> 
    - time(s): 643.505 
    - first_samp-last_samp: 0 - 643504
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7740 / 1000.0 = 7.74
2 - 310591 / 1000.0 = 310.591
3 - 320742 / 1000.0 = 320.742
4 - 632936 / 1000.0 = 632.936

Name event1 for sub-NETICU40_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU40_task-sedoffrest_eeg:no

______________________ sub-NETICU40_task-sedon1taken_eeg ______________________

    - Session 20240403 0733.mff 
    - 2024-04-03 12:33:33.810000+00:00 
    - <RawMff | signal1.bin, 135 x 643706 (643.7 s), ~663.2 MB, data loaded> 
    - time(s): 643.706 
    - first_samp-last_samp: 0 - 643705
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8121 / 1000.0 = 8.121
2 - 310800 / 1000.0 = 310.8
3 - 320955 / 1000.0 = 320.955
4 - 633149 / 1000.0 = 633.149
Name event1 for sub-NETICU40_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU40_task-sedon1taken_eeg:no
Name event3 for sub-NETICU40_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU40_task-sedon1taken_eeg:no

______________________ sub-NETICU40_task-sedon1rest_eeg ______________________

    - Session 20240403 0744

['CCTC07_RS_______________sub-CCTC07_task-rest_eeg',
 'CCTC07_ST_______________sub-CCTC07_task-taken_eeg',
 'CCTC07_IT_______________sub-CCTC07_task-taken_eeg',
 'CCTC07NETICUsedon1_RS_______________sub-CCTC07NETICU_task-sedon1rest_eeg',
 'CCTC07NETICUsedoff_RS_______________sub-CCTC07NETICU_task-sedoffrest_eeg',
 'CCTC07NETICUsedoff_ST_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedoff_IT_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedon1_ST_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'CCTC07NETICUsedon1_IT_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'MSICU32_RS_______________sub-MSICU32_task-rest_eeg',
 'MSICU32_ST_______________sub-MSICU32_task-taken_eeg',
 'MSICU32_IT_______________sub-MSICU32_task-taken_eeg',
 'MSICU38_ST_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_IT_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_RS_______________sub-MSICU38_task-rest_eeg',
 'MSICU42_RS_______________sub-MSICU42_

In [32]:
#####['NETICU31sedoff_RS_______________sub-NETICU31_task-sedon1rest_eeg']
var_names = []
for f in ['sub-NETICU31_task-sedon1rest_eeg']:
    print ('\n______________________',f,'______________________')
    ID = ((f[4:]).split("_task"))[0] # f[4:] removes 'sub-' from f, Then the .split() will split it every "_task", Then [0] picks the ID "CCTC03"
    source_path = f'./data/sub-{ID}/{f}'
    fname = (sorted(os.listdir(source_path)))[0]
    data_path = f'{source_path}/{fname}'
    # Loading data
    with io.capture_output() as captured:
        raw = mne.io.read_raw_egi(data_path, preload=True)
    data = raw.copy()
    raw_sfreq = raw.info['sfreq']

    #Timing############################################################################################
    meas_data = data.info['meas_date']
    print(f'\n    - {fname}',
          f'\n    - {meas_data}',
          f'\n    - {data}',
          f'\n    - time(s): {(data.last_samp+1)/raw_sfreq}',
          f'\n    - first_samp-last_samp: {data.first_samp} - {data.last_samp}')
    seconds = (data.last_samp+1)/raw_sfreq
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    print("    - Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))
    #EVENTS############################################################################################
    triggers = []
    for i in data.ch_names:
        if not 'E' in i:
            if not 'sync' in i:
                if not 'STI' in i:
                    if not 'V' in i:
                        print (f"    - Trigger <  ch. {i}>")
                        triggers.append(i)
    with io.capture_output() as captured:
        events = mne.find_events(data, stim_channel='STI 014', initial_event=True)
    for i in range(events.shape[0]):
        vars()['event'+str(i+1)] = events[i,0]
        print (f'{i+1} - {events[i,0]} / {raw_sfreq} = {(events[i,0])/raw_sfreq}')

    #crop####################################################################################################
    for i in range(events.shape[0]):
        name = input(f'Name event{i+1} for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name event{i+1} for {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < event{i+1} for {f}\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            if '_smin' in name:
                name = (name[:-5])
                smin = input(f'> smin to replace event{i+1} start time (type "zoom" to plot a closer look):')
                if smin == 'zoom':
                    with io.capture_output() as captured:
                        data.plot(duration=10,
                                  n_channels=10,
                                  scalings=1000e-6, #20e-6`` (20µV)
                                  #show_options=True,
                                  #group_by='position',
                                  butterfly=False,
                                  title=f,
                                  clipping=None,
                                  block=True)
                    time.sleep(2)
                    smin = int(input(f'> smin to replace event{i+1} start time:'))
                else:
                    smin = int(smin)
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')
            else:
                vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=vars()['event'+str(i+1)], smax=vars()['event'+str(i+1)]+(312*raw_sfreq), include_tmax=False)
                var_names.append(f'{ID}{name}_______________{f}')# 15x(_______________)
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
    #if no triggers
    if len(events) == 0 and len(triggers) == 0:
        name = input(f'Name file for {f}:')
        if name == 'plot':
            with io.capture_output() as captured:
                data.plot(duration=200,
                          n_channels=50,
                          scalings=1000e-6, #20e-6`` (20µV)
                          #show_options=True,
                          #group_by='position',
                          butterfly=False,
                          title=f,
                          clipping=None,
                          block=True)
            time.sleep(2)
            name = input(f'Name for file {f}:')
        if name == 'exit':
            print ('\n\n________________________________________________________________________________\n________________________________________________________________________________\n    Saved Variables:')
            for var in var_names:
                print(f'    --> {var}')
            print (f'    Checkpoint < for {f} (no events)\n________________________________________________________________________________\n________________________________________________________________________________\n')
            sys.exit()
        if 'no' not in name:
            missingtrig_smin = int(input(f'> smin value (samplingrate={raw_sfreq}) for {f}:'))
            vars()[ID+name+'_______________'+f]= nk.mne_crop(data.copy(),smin=missingtrig_smin, smax=missingtrig_smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{name}_______________{f}')
        if '_RS1st' in name:
            nameid = (name.split('RS1st'))[0]
            smin = int(input(f'> smin for RS2nd in {f}:'))
            vars()[ID+nameid+'RS2nd_______________'+f]= nk.mne_crop(data.copy(),smin=smin, smax=smin+(312*raw_sfreq), include_tmax=False)
            var_names.append(f'{ID}{nameid}RS2nd_______________{f}')
var_names




______________________ sub-NETICU31_task-sedon1rest_eeg ______________________

    - Session 20240207 1746.mff 
    - 2024-02-07 22:46:42.815000+00:00 
    - <RawMff | signal1.bin, 131 x 865088 (865.1 s), ~864.8 MB, data loaded> 
    - time(s): 865.088 
    - first_samp-last_samp: 0 - 865087
    - Duration = 0hr : 14min : 25sec
Name file for sub-NETICU31_task-sedon1rest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU31_task-sedon1rest_eeg:538087


['NETICU31sedoff_RS_______________sub-NETICU31_task-sedon1rest_eeg']

In [33]:
var_names

['NETICU31sedoff_RS_______________sub-NETICU31_task-sedon1rest_eeg']

In [21]:
var_names.remove('MSICU47np_______________sub-MSICU47_task-taken_eeg')

## Downsample

In [34]:
len(var_names)

1

In [35]:
# Downsampling to 250Hz & renamed
downsampled_names=[]
for Vname in var_names:
    print (Vname)
    vars()[(Vname.split('_______________'))[0]] = vars()[Vname].copy()
    downsampled_names.append((Vname.split('_______________'))[0])
    SRate= vars()[(Vname.split('_______________'))[0]].info['sfreq']
    if SRate > 250:
        with io.capture_output() as captured:
            vars()[(Vname.split('_______________'))[0]].resample(250) 
        #print(f'    > {Vname} Data was downsampled to 250Hz from {raw_sfreq}')
    if SRate < 250: 
        print(f'    > Error: Data sampling frequency is {SRate}Hz.')
        answer = input('      - Do you want to continue(yes/no)?:')
        if answer.lower().startswith("y"):
            print("      - ok, carry on then")
        elif answer.lower().startswith("n"):
            print("      - ok, sayonnara")
            sys.exit()

# if raw.info['sfreq'] != 250:
#     raw = raw.resample(250)

NETICU31sedoff_RS_______________sub-NETICU31_task-sedon1rest_eeg


## Save Individual Files 

In [36]:
#make cutdata folder
if os.path.isdir('./cutdata')==False:
    os.makedirs('./cutdata')
#save
for V in downsampled_names:
    path = f'./cutdata/{V}'
    if not os.path.exists(path):
        os.makedirs(path)
        print (f'{path} path created')
    with io.capture_output() as captured:
        vars()[V].save(f'{path}/{V}.fif', overwrite=True)
        #vars()[V].save(f'{path}/{V}_eeg.fif', overwrite=True)

./cutdata/NETICU31sedoff_RS path created


In [28]:
## NEW DATA w/ EDITS ###
#make cutdata folder
if os.path.isdir('./cutdata')==False:
    os.makedirs('./cutdata')
#save
for V in downsampled_names:
    if V == 'NETICU33sedon1_RS1st':
        path = f'./cutdata/NETICU33sedon1_RS'
        if not os.path.exists(path):
            os.makedirs(path)
            print (f'{path} path created')
        with io.capture_output() as captured:
            vars()[V].save(f'{path}/NETICU33sedon1_RS.fif', overwrite=True)
    elif V == 'NETICU33sedon1_RS2nd':
        path = f'./cutdata/NETICU33sedoff_RS'
        if not os.path.exists(path):
            os.makedirs(path)
            print (f'{path} path created')
        with io.capture_output() as captured:
            vars()[V].save(f'{path}/NETICU33sedoff_RS.fif', overwrite=True)    
    elif V == 'NETICU46sedon1_RS1st':
        path = f'./cutdata/NETICU46sedon1_RS'
        if not os.path.exists(path):
            os.makedirs(path)
            print (f'{path} path created')
        with io.capture_output() as captured:
            vars()[V].save(f'{path}/NETICU46sedon1_RS.fif', overwrite=True)
    elif V == 'NETICU46sedon1_RS2nd':
        path = f'./cutdata/NETICU46sedoff_RS'
        if not os.path.exists(path):
            os.makedirs(path)
            print (f'{path} path created')
        with io.capture_output() as captured:
            vars()[V].save(f'{path}/NETICU46sedoff_RS.fif', overwrite=True)
    else:
        path = f'./cutdata/{V}'
        if not os.path.exists(path):
            os.makedirs(path)
            print (f'{path} path created')
        with io.capture_output() as captured:
            vars()[V].save(f'{path}/{V}.fif', overwrite=True)
            #vars()[V].save(f'{path}/{V}_eeg.fif', overwrite=True)

./cutdata/CCTC07_RS path created
./cutdata/CCTC07_ST path created
./cutdata/CCTC07_IT path created
./cutdata/CCTC07NETICUsedon1_RS path created
./cutdata/CCTC07NETICUsedoff_RS path created
./cutdata/CCTC07NETICUsedoff_ST path created
./cutdata/CCTC07NETICUsedoff_IT path created
./cutdata/CCTC07NETICUsedon1_ST path created
./cutdata/CCTC07NETICUsedon1_IT path created
./cutdata/MSICU32_RS path created
./cutdata/MSICU32_ST path created
./cutdata/MSICU32_IT path created
./cutdata/MSICU38_ST path created
./cutdata/MSICU38_IT path created
./cutdata/MSICU38_RS path created
./cutdata/MSICU42_RS path created
./cutdata/MSICU42_ST path created
./cutdata/MSICU42_IT path created
./cutdata/MSICU43_RS path created
./cutdata/MSICU43_ST path created
./cutdata/MSICU43_IT path created
./cutdata/MSICU47_RS path created
./cutdata/MSICU47_ST path created
./cutdata/MSICU47_IT path created
./cutdata/NETICU31sedoff_ST path created
./cutdata/NETICU31sedoff_IT path created
./cutdata/NETICU31sedon2_RS path create

## Load .fif Files

In [29]:
fnames = sorted(os.listdir('./cutdata'))
for i in fnames:
    with io.capture_output() as captured:
        raw = mne.io.read_raw_fif(f'./cutdata/{i}/{i}.fif', preload=True)
    print (i, raw.info['sfreq'])
    vars()[i] = raw

CCTC03_IT 250.0
CCTC03_RS 250.0
CCTC03_ST 250.0
CCTC03part2_IT 250.0
CCTC03part2_RS 250.0
CCTC03part2_ST 250.0
CCTC04_IT 250.0
CCTC04_RS 250.0
CCTC04_RSredo 250.0
CCTC04_ST 250.0
CCTC05_IT 250.0
CCTC05_RS 250.0
CCTC05_ST 250.0
CCTC07NETICUsedoff_IT 250.0
CCTC07NETICUsedoff_RS 250.0
CCTC07NETICUsedoff_ST 250.0
CCTC07NETICUsedon1_IT 250.0
CCTC07NETICUsedon1_RS 250.0
CCTC07NETICUsedon1_ST 250.0
CCTC07_IT 250.0
CCTC07_RS 250.0
CCTC07_ST 250.0
HC01_IT 250.0
HC01_ST 250.0
HC02_IT 250.0
HC02_ST 250.0
HC03_IT 250.0
HC03_ST 250.0
HC04_IT 250.0
HC04_ST 250.0
HC05_IT1st 250.0
HC05_IT2nd 250.0
HC05_ST 250.0
HC06_IT 250.0
HC06_ST 250.0
HC07_IT 250.0
HC07_ST 250.0
HC08_IT 250.0
HC08_ST 250.0
HC09_IT 250.0
HC09_ST 250.0
HC10_IT 250.0
HC10_ST 250.0
HC11_IT 250.0
HC11_ST 250.0
HC12_IT 250.0
HC12_ST 250.0
HC13_IT 250.0
HC13_ST 250.0
HC14_IT 250.0
HC14_ST 250.0
HC15_IT 250.0
HC15_ST 250.0
HC16_IT 250.0
HC16_ST 250.0
HC17_IT 250.0
HC17_ST 250.0
HC18_IT 250.0
HC18_ST 250.0
MSICU05_IT 250.0
MSICU05_RS 250.0

In [ ]:
#Total file amt. for MIMC = 64 (upto MSICU30)
#Total file amt. for NETICU = 119 (upto NETICU30)
_________________________________________________

#Total files = 183

In [ ]:
# *220 (or 219) w/ HC files

In [14]:
for V in downsampled_names:
    print (f"'{V}',")

'HC01_ST',
'HC01_IT',
'HC02_ST',
'HC02_IT',
'HC03_IT',
'HC03_ST',
'HC04_ST',
'HC04_IT',
'HC05_ST',
'HC05_IT1st',
'HC05_IT2nd',
'HC06_ST',
'HC06_IT',
'HC07_ST',
'HC07_IT',
'HC08_ST',
'HC08_IT',
'HC09_ST',
'HC09_IT',
'HC10_ST',
'HC10_IT',
'HC11_ST',
'HC11_IT',
'HC12_ST',
'HC12_IT',
'HC13_ST',
'HC13_IT',
'HC14_ST',
'HC14_IT',
'HC15_ST',
'HC15_IT',
'HC16_ST',
'HC16_IT',
'HC17_ST',
'HC17_IT',
'HC18_ST',
'HC18_IT',


In [ ]:
## NEW DATA ##

#Total file amt. for MIMC = 82 (upto MSICU47)
#Total file amt. for NETICU = 157 (upto NETICU46)
_________________________________________________

#Total files = 239

#or 276 w/HC

# =============================================================

# ===========================EXTRA============================

# =============================================================


In [43]:
for i in sorted(os.listdir('../sourcedata')):
    if 'HC' in i:
        for c in sorted(os.listdir(f'../sourcedata/{i}/eeg')):
            print (i, sorted(os.listdir(f'../sourcedata/{i}/eeg/{c}')))



sub-HC01 ['Session 20160308 1514.zip']
sub-HC01 ['Session 20161123 1429.zip']
sub-HC02 ['Session 20160520 1242.zip']
sub-HC03 ['Session 20160524 0942.zip']
sub-HC04 ['Session 20160527 1145.zip']
sub-HC05 ['Session 20160530 1318.zip']
sub-HC06 ['Session 20160615 1350.zip']
sub-HC07 ['Session 20160617 1342.zip']
sub-HC08 ['Session 20160617 1618.zip']
sub-HC09 ['Session 20160621 1415.zip']
sub-HC10 ['Session 20160622 1550.zip']
sub-HC11 ['Session 20160624 1158.zip']
sub-HC12 ['Session 20160624 1319.zip']
sub-HC13 ['Session 20160705 1347.zip']
sub-HC14 ['Session 20160706 1321.zip']
sub-HC15 ['Session 20161026 1322.zip']
sub-HC16 ['Session 20170315 1317.zip']
sub-HC17 ['Session 20170329 1426.zip']
sub-HC18 ['Session 20170330 1425.zip']


steps:
- File format
    - HC
    - GOSE for patients
        - labels for each participant for the ML step
make this one step incase I want to change hyperparameters (like what electrodes to choose, and how to preproc
- preproc
- Create a Database
- feature extraction (start wit SVD Complexity)
- choose electrodes to average
- add labels
- run algorithms
- 

In [54]:
#New folder (downsampling and cropping data)
#matplotlib.use('Qt5Agg')
for f in fol_names:
    print ('\n______________________',f,'______________________')
    ID = ((f[4:]).split("_task"))[0] # f[4:] removes 'sub-' from f, Then the .split() will split it every "_task", Then [0] picks the ID "CCTC03"
    source_path = f'./data/sub-{ID}/{f}'
    fname = (sorted(os.listdir(source_path)))[0]
    data_path = f'{source_path}/{fname}'
    print (fname)
    # Loading metadata
    with io.capture_output() as captured:
        raw = mne.io.read_raw_egi(data_path, preload=True)
    data = raw.copy()       
    # Downsampling to 250Hz
    raw_sfreq = raw.info['sfreq']
#     data_sfreq = data.info['sfreq']
#     if data_sfreq > 250:
#         with io.capture_output() as captured:
#             data.resample(250)
#         print(f'    > Data was downsampled to 250Hz from {raw_sfreq}')
#     if data_sfreq < 250: 
#         print(f'    > Error: Data sampling frequency is {raw_sfreq}Hz.')
#         answer = input('      - Do you want to continue(yes/no)?:')
#         if answer.lower().startswith("y"):
#             print("      - ok, carry on then")
#         elif answer.lower().startswith("n"):
#             print("      - ok, sayonnara")
#             sys.exit()
#     # loading in data      
#     with io.capture_output() as captured: 
#         data.load_data() 

    


______________________ sub-CCTC03_task-taken_eeg ______________________
Session 20010114 0336.mff
(1) Data timing... 
    - Session 20010114 0336.mff ----> 2001-01-14 08:44:31.092000+00:00 
    - <RawMff | signal1.bin, 133 x 171170 (684.7 s), ~173.9 MB, data loaded> 
    - time(s): 684.68 
    - first_samp-last_samp: 0 - 171169
    - Duration = 0hr : 11min : 24sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 964 / 250.0 = 3.856
 - 89705 / 250.0 = 358.82

______________________ sub-CCTC03_task-rest02_eeg ______________________
Session 20010118 0013.mff
(1) Data timing... 
    - Session 20010118 0013.mff ----> 2001-01-18 05:23:08.496000+00:00 
    - <RawMff | signal1.bin, 132 x 97510 (390.0 s), ~98.4 MB, data loaded> 
    - time(s): 390.04 
    - first_samp-last_samp: 0 - 97509
    - Duration = 0hr : 06min : 30sec
(2) Trigger Channels
    - Trigger ch. 12  
 - 1499 / 250.0 = 5.996

______________________ sub-CCTC03_task-taken02_eeg ______________________
Session

(1) Data timing... 
    - Session 20160621 1415.mff ----> 2016-06-21 19:33:12.100000+00:00 
    - <RawMff | signal1.bin, 134 x 304871 (1219.5 s), ~311.9 MB, data loaded> 
    - time(s): 1219.484 
    - first_samp-last_samp: 0 - 304870
    - Duration = 0hr : 20min : 19sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2640 / 250.0 = 10.56
 - 107138 / 250.0 = 428.552
 - 211844 / 250.0 = 847.376

______________________ sub-HC10_task-Taken_eeg ______________________
Session 20160622 1550.mff
(1) Data timing... 
    - Session 20160622 1550.mff ----> 2016-06-22 21:04:32.204000+00:00 
    - <RawMff | signal1.bin, 134 x 312207 (1248.8 s), ~319.4 MB, data loaded> 
    - time(s): 1248.828 
    - first_samp-last_samp: 0 - 312206
    - Duration = 0hr : 20min : 48sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2646 / 250.0 = 10.584
 - 118606 / 250.0 = 474.424
 - 219704 / 250.0 = 878.816

_________________

(1) Data timing... 
    - Session 20220727 0858.mff ----> 2022-07-27 14:09:49.536000+00:00 
    - <RawMff | signal1.bin, 132 x 90123 (360.5 s), ~90.9 MB, data loaded> 
    - time(s): 360.492 
    - first_samp-last_samp: 0 - 90122
    - Duration = 0hr : 06min : 00sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1523 / 250.0 = 6.092

______________________ sub-MSICU10_task-taken_eeg ______________________
Session 20220727 0926.mff
(1) Data timing... 
    - Session 20220727 0926.mff ----> 2022-07-27 14:32:38.148000+00:00 
    - <RawMff | signal1.bin, 133 x 206319 (825.3 s), ~209.5 MB, data loaded> 
    - time(s): 825.276 
    - first_samp-last_samp: 0 - 206318
    - Duration = 0hr : 13min : 45sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
 - 1254 / 250.0 = 5.016
 - 8803 / 250.0 = 35.212
 - 124083 / 250.0 = 496.332

______________________ sub-MSICU10_task-scrambled_eeg ______________________
Session 20220727 0916.mff
(1) Data timing... 
    - Session 20220727 0916

(1) Data timing... 
    - Session 20230609 1229.mff ----> 2023-06-09 17:56:09.966000+00:00 
    - <RawMff | signal1.bin, 137 x 978030 (978.0 s), ~1022.4 MB, data loaded> 
    - time(s): 978.03 
    - first_samp-last_samp: 0 - 978029
    - Duration = 0hr : 16min : 18sec
(2) Trigger Channels
    - Trigger ch. 59  
    - Trigger ch. 69  
    - Trigger ch. 62  
    - Trigger ch. 72  
    - Trigger ch. 60  
    - Trigger ch. 70  
 - 6207 / 1000.0 = 6.207
 - 336296 / 1000.0 = 336.296
 - 346461 / 1000.0 = 346.461
 - 649124 / 1000.0 = 649.124
 - 659303 / 1000.0 = 659.303
 - 971501 / 1000.0 = 971.501

______________________ sub-MSICU30_task-RS_ST_Taken_eeg ______________________
Session 20230614 1437.mff
(1) Data timing... 
    - Session 20230614 1437.mff ----> 2023-06-14 19:52:12.138000+00:00 
    - <RawMff | signal1.bin, 137 x 977979 (978.0 s), ~1022.4 MB, data loaded> 
    - time(s): 977.979 
    - first_samp-last_samp: 0 - 977978
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
   

(1) Data timing... 
    - Session 20210217 1130.mff ----> 2021-02-17 16:32:26.357000+00:00 
    - <RawMff | signal1.bin, 131 x 610578 (610.6 s), ~610.4 MB, data loaded> 
    - time(s): 610.578 
    - first_samp-last_samp: 0 - 610577
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels

______________________ sub-NETICU05_task-sedon1taken_eeg ______________________
Session 20210217 1037.mff
(1) Data timing... 
    - Session 20210217 1037.mff ----> 2021-02-17 15:43:32.004000+00:00 
    - <RawMff | signal1.bin, 133 x 306437 (1225.7 s), ~311.1 MB, data loaded> 
    - time(s): 1225.748 
    - first_samp-last_samp: 0 - 306436
    - Duration = 0hr : 20min : 25sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1557 / 250.0 = 6.228
 - 228342 / 250.0 = 913.368

______________________ sub-NETICU05_task-sedon1rest_eeg ______________________
Session 20210217 1014.mff
(1) Data timing... 
    - Session 20210217 1014.mff ----> 2021-02-17 15:26:23.850000+00:00 
    - <RawMff

(1) Data timing... 
    - Session 20220329 0734.mff ----> 2022-03-29 12:48:11.424000+00:00 
    - <RawMff | signal1.bin, 131 x 631678 (631.7 s), ~631.5 MB, data loaded> 
    - time(s): 631.678 
    - first_samp-last_samp: 0 - 631677
    - Duration = 0hr : 10min : 31sec
(2) Trigger Channels

______________________ sub-NETICU15_task-sedon1rest_eeg ______________________
Session 20220526 1730.mff
(1) Data timing... 
    - Session 20220526 1730.mff ----> 2022-05-26 22:32:00.072000+00:00 
    - <RawMff | signal1.bin, 131 x 568979 (569.0 s), ~568.8 MB, data loaded> 
    - time(s): 568.979 
    - first_samp-last_samp: 0 - 568978
    - Duration = 0hr : 09min : 28sec
(2) Trigger Channels

______________________ sub-NETICU15_task-sedon1taken_eeg ______________________
Session 20220526 1701.mff
(1) Data timing... 
    - Session 20220526 1701.mff ----> 2022-05-26 22:08:45.408000+00:00 
    - <RawMff | signal1.bin, 134 x 185544 (742.2 s), ~189.9 MB, data loaded> 
    - time(s): 742.176 
    - first

(1) Data timing... 
    - Session 20010322 0427.mff ----> 2001-03-22 09:37:18.005000+00:00 
    - <RawMff | signal1.bin, 133 x 999764 (999.8 s), ~1014.6 MB, data loaded> 
    - time(s): 999.764 
    - first_samp-last_samp: 0 - 999763
    - Duration = 0hr : 16min : 39sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 3747 / 1000.0 = 3.747
 - 295382 / 1000.0 = 295.382
 - 667432 / 1000.0 = 667.432

______________________ sub-NETICU22_task-sedon1rest_eeg ______________________
Session 20010322 0459.mff
(1) Data timing... 
    - Session 20010322 0459.mff ----> 2001-03-22 10:08:23.525000+00:00 
    - <RawMff | signal1.bin, 132 x 631179 (631.2 s), ~635.8 MB, data loaded> 
    - time(s): 631.179 
    - first_samp-last_samp: 0 - 631178
    - Duration = 0hr : 10min : 31sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 3736 / 1000.0 = 3.736

______________________ sub-NETICU22_task-sedoffrest_eeg ______________________
Session 20010322 0525.mff
(1) Data timing... 
    - Se

(1) Data timing... 
    - Session 20230426 1336.mff ----> 2023-04-26 18:37:50.704000+00:00 
    - <RawMff | signal1.bin, 135 x 642304 (642.3 s), ~661.7 MB, data loaded> 
    - time(s): 642.304 
    - first_samp-last_samp: 0 - 642303
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 6944 / 1000.0 = 6.944
 - 309419 / 1000.0 = 309.419
 - 319609 / 1000.0 = 319.609
 - 631802 / 1000.0 = 631.802

______________________ sub-NETICU28_task-sedoffrest_eeg ______________________
Session 20230426 1422.mff
(1) Data timing... 
    - Session 20230426 1422.mff ----> 2023-04-26 19:25:24.451000+00:00 
    - <RawMff | signal1.bin, 133 x 611507 (611.5 s), ~620.7 MB, data loaded> 
    - time(s): 611.507 
    - first_samp-last_samp: 0 - 611506
    - Duration = 0hr : 10min : 11sec
(2) Trigger Channels
    - Trigger ch. 18  
    - Trigger ch. 28  
 - 6055 / 1000.0 = 6.055
 - 606089 / 1000.0 = 606.089

_______

# Save figures

In [ ]:
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}'):
    os.makedirs(f'./{project}/out_figures/sub-{ID}/task-{task}')
    print('new directory made')

#data
step = 'data'
data.info['bads'].extend(['Vertex Reference'])
data.compute_psd(picks='eeg', fmax=70).plot()
data.info['bads'].remove('Vertex Reference')
data.info['bads']
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')

#bads
step = 'data_bads'
data_bads.info['bads'].extend(['Vertex Reference'])
data_bads.compute_psd(picks='eeg', fmax=70).plot()
data_bads.info['bads'].remove('Vertex Reference')
data_bads.info['bads']
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')
@#saving marked_bads.txt
if not os.path.exists(f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt'):
    open(f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'x')
    with open (f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'w') as badsfile:
        badsfile.write('\n'.join(marked_bad))
        print('.txt file created')
else:
    with open (f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'w') as outfile:
        outfile.write('\n'.join(marked_bad))
        print('marked_bads.txt was updated')

        
#reref
step = 'data_bads_filter_reref'
data_bads_filter_reref.compute_psd(picks='eeg', fmax=70).plot()
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')



In [ ]:

        print ("    > Rest Start (s):", trig09, "<")
        print ("    > Rest End (s):", trig19, "<")
        print ("    > Scrambled Start (s):", trig12, "<")
        print ("    > Scrambled End (s):", trig22, "<")
        print ("    > Taken Start (s):", trig10, "<")
        print ("    > Taken End (s):", trig20, "<")
        print ("    > ERROR idk what trigger 11 is:", trig11, "< (for HC, this was a second viewing of intact taken)")
        print ("    > ERROR idk what trigger 99 is:", trig99, "<")        
 



In [ ]:
______________________ sub-MSICU10_task-taken_eeg ______________________

    - Session 20220727 0926.mff 
    - 2022-07-27 14:32:38.148000+00:00 
    - <RawMff | signal1.bin, 133 x 206319 (825.3 s), ~209.5 MB, data loaded> 
    - time(s): 825.276 
    - first_samp-last_samp: 0 - 206318
    - Duration = 0hr : 13min : 45sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  > again
1 - 1254 / 250.0 = 5.016
2 - 8803 / 250.0 = 35.212
3 - 124083 / 250.0 = 496.332
Name event1 for sub-MSICU10_task-taken_eeg:plot
Name event1 for sub-MSICU10_task-taken_eeg:no
Name event2 for sub-MSICU10_task-taken_eeg:_ST
Name event3 for sub-MSICU10_task-taken_eeg:_IT

    
    
    
    
______________________ sub-HC05_task-Taken_eeg ______________________

    - Session 20160530 1318.mff 
    - 2016-05-30 18:36:37.464000+00:00 
    - <RawMff | signal1.bin, 134 x 456930 (1827.7 s), ~467.3 MB, data loaded> 
    - time(s): 1827.72 
    - first_samp-last_samp: 0 - 456929
    - Duration = 0hr : 30min : 27sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  > again
    - Trigger <  ch. 11  >
1 - 2584 / 250.0 = 10.336
2 - 130584 / 250.0 = 522.336
3 - 249684 / 250.0 = 998.736
4 - 357717 / 250.0 = 1430.868
Name event1 for sub-HC05_task-Taken_eeg:plot
Name event1 for sub-HC05_task-Taken_eeg:_ST
Name event2 for sub-HC05_task-Taken_eeg:_IT1st
Name event3 for sub-HC05_task-Taken_eeg:_IT2nd
Name event4 for sub-HC05_task-Taken_eeg:no

In [ ]:









_______
________________________________________________________________________________



## New data

In [3]:




####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

______________________ sub-HC01_task-ST_eeg ______________________

    - Session 20161123 1429.mff 
    - 2016-11-23 19:49:42.096000+00:00 
    - <RawMff | signal1.bin, 133 x 228821 (915.3 s), ~232.4 MB, data loaded> 
    - time(s): 915.284 
    - first_samp-last_samp: 0 - 228820
    - Duration = 0hr : 15min : 15sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
1 - 3036 / 250.0 = 12.144
2 - 128092 / 250.0 = 512.368
Name event1 for sub-HC01_task-ST_eeg:plot
Name event1 for sub-HC01_task-ST_eeg:no
Name event2 for sub-HC01_task-ST_eeg:_ST

______________________ sub-HC01_task-IT_eeg ______________________

    - Session 20160308 1514.mff 
    - 2016-03-08 20:42:37.112000+00:00 
    - <RawMff | signal1.bin, 133 x 357782 (1431.1 s), ~363.2 MB, data loaded> 
    - time(s): 1431.128 
    - first_samp-last_samp: 0 - 357781
    - Duration = 0hr : 23min : 51sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2432 / 250.0 = 9.728
2 - 272682 / 250.0 = 1090.728
Name event1 for sub-HC01_task-IT_eeg:plot
Name event1 for sub-HC01_task-IT_eeg:_IT
Name event2 for sub-HC01_task-IT_eeg:no

______________________ sub-HC02_task-Taken_eeg ______________________

    - Session 20160520 1242.mff 
    - 2016-05-20 17:44:59.212000+00:00 
    - <RawMff | signal1.bin, 133 x 262309 (1049.2 s), ~266.3 MB, data loaded> 
    - time(s): 1049.236 
    - first_samp-last_samp: 0 - 262308
    - Duration = 0hr : 17min : 29sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2310 / 250.0 = 9.24
2 - 160262 / 250.0 = 641.048
Name event1 for sub-HC02_task-Taken_eeg:_ST
Name event2 for sub-HC02_task-Taken_eeg:_IT

______________________ sub-HC03_task-Taken_eeg ______________________

    - Session 20160524 0942.mff 
    - 2016-05-24 15:14:19.604000+00:00 
    - <RawMff | signal1.bin, 134 x 404568 (1618.3 s), ~413.8 MB, data loaded> 
    - time(s): 1618.272 
    - first_samp-last_samp: 0 - 404567
    - Duration = 0hr : 26min : 58sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
    - Trigger <  ch. 12  >
1 - 2831 / 250.0 = 11.324
2 - 178964 / 250.0 = 715.856
3 - 309102 / 250.0 = 1236.408
Name event1 for sub-HC03_task-Taken_eeg:_IT
Name event2 for sub-HC03_task-Taken_eeg:no
Name event3 for sub-HC03_task-Taken_eeg:_ST

______________________ sub-HC04_task-Taken_eeg ______________________

    - Session 20160527 1145.mff 
    - 2016-05-27 16:49:10.816000+00:00 
    - <RawMff | signal1.bin, 134 x 299920 (1199.7 s), ~306.8 MB, data loaded> 
    - time(s): 1199.68 
    - first_samp-last_samp: 0 - 299919
    - Duration = 0hr : 19min : 59sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2349 / 250.0 = 9.396
2 - 118235 / 250.0 = 472.94
3 - 211461 / 250.0 = 845.844
Name event1 for sub-HC04_task-Taken_eeg:_ST
Name event2 for sub-HC04_task-Taken_eeg:_IT
Name event3 for sub-HC04_task-Taken_eeg:no

______________________ sub-HC05_task-Taken_eeg ______________________

    - Session 20160530 1318.mff 
    - 2016-05-30 18:36:37.464000+00:00 
    - <RawMff | signal1.bin, 134 x 456930 (1827.7 s), ~467.3 MB, data loaded> 
    - time(s): 1827.72 
    - first_samp-last_samp: 0 - 456929
    - Duration = 0hr : 30min : 27sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  > again
    - Trigger <  ch. 11  >
1 - 2584 / 250.0 = 10.336
2 - 130584 / 250.0 = 522.336
3 - 249684 / 250.0 = 998.736
4 - 357717 / 250.0 = 1430.868
Name event1 for sub-HC05_task-Taken_eeg:plot
Name event1 for sub-HC05_task-Taken_eeg:_ST
Name event2 for sub-HC05_task-Taken_eeg:_IT1st
Name event3 for sub-HC05_task-Taken_eeg:_IT2nd
Name event4 for sub-HC05_task-Taken_eeg:no

______________________ sub-HC06_task-Taken_eeg ______________________

    - Session 20160615 1350.mff 
    - 2016-06-15 19:04:22.716000+00:00 
    - <RawMff | signal1.bin, 134 x 331370 (1325.5 s), ~339.0 MB, data loaded> 
    - time(s): 1325.48 
    - first_samp-last_samp: 0 - 331369
    - Duration = 0hr : 22min : 05sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2621 / 250.0 = 10.484
2 - 126705 / 250.0 = 506.82
3 - 235987 / 250.0 = 943.948
Name event1 for sub-HC06_task-Taken_eeg:_ST
Name event2 for sub-HC06_task-Taken_eeg:_IT
Name event3 for sub-HC06_task-Taken_eeg:no

______________________ sub-HC07_task-Taken_eeg ______________________

    - Session 20160617 1342.mff 
    - 2016-06-17 19:04:57.788000+00:00 
    - <RawMff | signal1.bin, 134 x 310421 (1241.7 s), ~317.5 MB, data loaded> 
    - time(s): 1241.684 
    - first_samp-last_samp: 0 - 310420
    - Duration = 0hr : 20min : 41sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2665 / 250.0 = 10.66
2 - 111296 / 250.0 = 445.184
3 - 214258 / 250.0 = 857.032
Name event1 for sub-HC07_task-Taken_eeg:_ST
Name event2 for sub-HC07_task-Taken_eeg:_IT
Name event3 for sub-HC07_task-Taken_eeg:no

______________________ sub-HC08_task-Taken_eeg ______________________

    - Session 20160617 1618.mff 
    - 2016-06-17 21:43:54.144000+00:00 
    - <RawMff | signal1.bin, 134 x 309482 (1237.9 s), ~316.6 MB, data loaded> 
    - time(s): 1237.928 
    - first_samp-last_samp: 0 - 309481
    - Duration = 0hr : 20min : 37sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2033 / 250.0 = 8.132
2 - 110966 / 250.0 = 443.864
3 - 215739 / 250.0 = 862.956
Name event1 for sub-HC08_task-Taken_eeg:_ST
Name event2 for sub-HC08_task-Taken_eeg:_IT
Name event3 for sub-HC08_task-Taken_eeg:no

______________________ sub-HC09_task-Taken_eeg ______________________

    - Session 20160621 1415.mff 
    - 2016-06-21 19:33:12.100000+00:00 
    - <RawMff | signal1.bin, 134 x 304871 (1219.5 s), ~311.9 MB, data loaded> 
    - time(s): 1219.484 
    - first_samp-last_samp: 0 - 304870
    - Duration = 0hr : 20min : 19sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2640 / 250.0 = 10.56
2 - 107138 / 250.0 = 428.552
3 - 211844 / 250.0 = 847.376
Name event1 for sub-HC09_task-Taken_eeg:_ST
Name event2 for sub-HC09_task-Taken_eeg:_IT
Name event3 for sub-HC09_task-Taken_eeg:no

______________________ sub-HC10_task-Taken_eeg ______________________

    - Session 20160622 1550.mff 
    - 2016-06-22 21:04:32.204000+00:00 
    - <RawMff | signal1.bin, 134 x 312207 (1248.8 s), ~319.4 MB, data loaded> 
    - time(s): 1248.828 
    - first_samp-last_samp: 0 - 312206
    - Duration = 0hr : 20min : 48sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2646 / 250.0 = 10.584
2 - 118606 / 250.0 = 474.424
3 - 219704 / 250.0 = 878.816
Name event1 for sub-HC10_task-Taken_eeg:_ST
Name event2 for sub-HC10_task-Taken_eeg:_IT
Name event3 for sub-HC10_task-Taken_eeg:no

______________________ sub-HC11_task-Taken_eeg ______________________

    - Session 20160624 1158.mff 
    - 2016-06-24 17:18:48.496000+00:00 
    - <RawMff | signal1.bin, 134 x 306232 (1224.9 s), ~313.3 MB, data loaded> 
    - time(s): 1224.928 
    - first_samp-last_samp: 0 - 306231
    - Duration = 0hr : 20min : 24sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2694 / 250.0 = 10.776
2 - 108052 / 250.0 = 432.208
3 - 213503 / 250.0 = 854.012
Name event1 for sub-HC11_task-Taken_eeg:_ST
Name event2 for sub-HC11_task-Taken_eeg:_IT
Name event3 for sub-HC11_task-Taken_eeg:no

______________________ sub-HC12_task-Taken_eeg ______________________

    - Session 20160624 1319.mff 
    - 2016-06-24 18:34:27.356000+00:00 
    - <RawMff | signal1.bin, 134 x 315995 (1264.0 s), ~323.2 MB, data loaded> 
    - time(s): 1263.98 
    - first_samp-last_samp: 0 - 315994
    - Duration = 0hr : 21min : 03sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2039 / 250.0 = 8.156
2 - 115728 / 250.0 = 462.912
3 - 222752 / 250.0 = 891.008
Name event1 for sub-HC12_task-Taken_eeg:_ST
Name event2 for sub-HC12_task-Taken_eeg:_IT
Name event3 for sub-HC12_task-Taken_eeg:no


______________________ sub-HC13_task-Taken_eeg ______________________

    - Session 20160705 1347.mff 
    - 2016-07-05 19:02:45.400000+00:00 
    - <RawMff | signal1.bin, 134 x 310319 (1241.3 s), ~317.4 MB, data loaded> 
    - time(s): 1241.276 
    - first_samp-last_samp: 0 - 310318
    - Duration = 0hr : 20min : 41sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2702 / 250.0 = 10.808
2 - 106453 / 250.0 = 425.812
3 - 217384 / 250.0 = 869.536
Name event1 for sub-HC13_task-Taken_eeg:_ST
Name event2 for sub-HC13_task-Taken_eeg:_IT
Name event3 for sub-HC13_task-Taken_eeg:no

______________________ sub-HC14_task-Taken_eeg ______________________

    - Session 20160706 1321.mff 
    - 2016-07-06 18:32:17.620000+00:00 
    - <RawMff | signal1.bin, 134 x 300683 (1202.7 s), ~307.6 MB, data loaded> 
    - time(s): 1202.732 
    - first_samp-last_samp: 0 - 300682
    - Duration = 0hr : 20min : 02sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 11  >
1 - 2397 / 250.0 = 9.588
2 - 105013 / 250.0 = 420.052
3 - 207642 / 250.0 = 830.568
Name event1 for sub-HC14_task-Taken_eeg:_ST
Name event2 for sub-HC14_task-Taken_eeg:_IT
Name event3 for sub-HC14_task-Taken_eeg:no

______________________ sub-HC15_task-Taken_eeg ______________________

    - Session 20161026 1322.mff 
    - 2016-10-26 18:47:34.780000+00:00 
    - <RawMff | signal1.bin, 133 x 222409 (889.6 s), ~225.9 MB, data loaded> 
    - time(s): 889.636 
    - first_samp-last_samp: 0 - 222408
    - Duration = 0hr : 14min : 49sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2685 / 250.0 = 10.74
2 - 124834 / 250.0 = 499.336
Name event1 for sub-HC15_task-Taken_eeg:_ST
Name event2 for sub-HC15_task-Taken_eeg:_IT

______________________ sub-HC16_task-Taken_eeg ______________________

    - Session 20170315 1317.mff 
    - 2017-03-15 18:37:41.816000+00:00 
    - <RawMff | signal1.bin, 133 x 219367 (877.5 s), ~222.8 MB, data loaded> 
    - time(s): 877.468 
    - first_samp-last_samp: 0 - 219366
    - Duration = 0hr : 14min : 37sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2333 / 250.0 = 9.332
2 - 119100 / 250.0 = 476.4
Name event1 for sub-HC16_task-Taken_eeg:_ST
Name event2 for sub-HC16_task-Taken_eeg:_IT

______________________ sub-HC17_task-Taken_eeg ______________________

    - Session 20170329 1426.mff 
    - 2017-03-29 19:43:25.788000+00:00 
    - <RawMff | signal1.bin, 133 x 247105 (988.4 s), ~250.9 MB, data loaded> 
    - time(s): 988.42 
    - first_samp-last_samp: 0 - 247104
    - Duration = 0hr : 16min : 28sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2769 / 250.0 = 11.076
2 - 152691 / 250.0 = 610.764
Name event1 for sub-HC17_task-Taken_eeg:_ST
Name event2 for sub-HC17_task-Taken_eeg:_IT

______________________ sub-HC18_task-Taken_eeg ______________________

    - Session 20170330 1425.mff 
    - 2017-03-30 19:40:47.532000+00:00 
    - <RawMff | signal1.bin, 133 x 227193 (908.8 s), ~230.7 MB, data loaded> 
    - time(s): 908.772 
    - first_samp-last_samp: 0 - 227192
    - Duration = 0hr : 15min : 08sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2618 / 250.0 = 10.472
2 - 136853 / 250.0 = 547.412
Name event1 for sub-HC18_task-Taken_eeg:_ST
Name event2 for sub-HC18_task-Taken_eeg:_IT


####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

# ________________________________________________________________________________________________
# ____________________________________________CCTC03______________________________________________
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20010114 0336 ----> 2001-01-14 08:44:31.092000+00:00 
#     - <RawMff | signal1.bin, 133 x 171170 (684.7 s), ~173.9 MB, data loaded> 
#     - time(s): 684.68 
#     - first_samp-last_samp: 0 - 171169
#     - Duration = 0hr : 11min : 24sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 964 / 250 = 3.856
#  - 89705 / 250 = 358.82
# ___________ rest02 ___________
# (1) Data timing... 
#     - Session 20010118 0013 ----> 2001-01-18 05:23:08.496000+00:00 
#     - <RawMff | signal1.bin, 132 x 97510 (390.0 s), ~98.4 MB, data loaded> 
#     - time(s): 390.04 
#     - first_samp-last_samp: 0 - 97509
#     - Duration = 0hr : 06min : 30sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#  - 1499 / 250 = 5.996
# ___________ taken02 ___________
# (1) Data timing... 
#     - Session 20010118 0046 ----> 2001-01-18 05:48:01.580000+00:00 
#     - <RawMff | signal1.bin, 133 x 170857 (683.4 s), ~173.5 MB, data loaded> 
#     - time(s): 683.428 
#     - first_samp-last_samp: 0 - 170856
#     - Duration = 0hr : 11min : 23sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 949 / 250 = 3.796
#  - 89670 / 250 = 358.68
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20010114 0307 ----> 2001-01-14 08:20:31.100000+00:00 
#     - <RawMff | signal1.bin, 131 x 99797 (399.2 s), ~99.9 MB, data loaded> 
#     - time(s): 399.188 
#     - first_samp-last_samp: 0 - 99796
#     - Duration = 0hr : 06min : 39sec
# (2) Trigger Channels

# ________________________________________________________________________________________________
# ____________________________________________CCTC04______________________________________________
# ___________ restredo ___________
# (1) Data timing... 
#     - Session 20010219 0100 ----> 2001-02-19 06:04:58.056000+00:00 
#     - <RawMff | signal1.bin, 132 x 97572 (390.3 s), ~98.4 MB, data loaded> 
#     - time(s): 390.288 
#     - first_samp-last_samp: 0 - 97571
#     - Duration = 0hr : 06min : 30sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1006 / 250 = 4.024
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20010219 0042 ----> 2001-02-19 05:44:29.564000+00:00 
#     - <RawMff | signal1.bin, 133 x 168720 (674.9 s), ~171.4 MB, data loaded> 
#     - time(s): 674.88 
#     - first_samp-last_samp: 0 - 168719
#     - Duration = 0hr : 11min : 14sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 965 / 250 = 3.86
#  - 86863 / 250 = 347.452
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20010219 0028 ----> 2001-02-19 05:32:18.128000+00:00 
#     - <RawMff | signal1.bin, 132 x 97659 (390.6 s), ~98.5 MB, data loaded> 
#     - time(s): 390.636 
#     - first_samp-last_samp: 0 - 97658
#     - Duration = 0hr : 06min : 30sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#  - 1518 / 250 = 6.072

# ________________________________________________________________________________________________
# ____________________________________________CCTC05______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230516 1650 ----> 2023-05-16 21:59:33.873000+00:00 
#     - <RawMff | signal1.bin, 137 x 244545 (978.2 s), ~255.8 MB, data loaded> 
#     - time(s): 978.18 
#     - first_samp-last_samp: 0 - 244544
#     - Duration = 0hr : 16min : 18sec
# (2) Trigger Channels
#     - Trigger ch. 59  
#     - Trigger ch. 69  
#     - Trigger ch. 62  
#     - Trigger ch. 72  
#     - Trigger ch. 60  
#     - Trigger ch. 70  
#  - 1609 / 250 = 6.436
#  - 84128 / 250 = 336.512
#  - 86666 / 250 = 346.664
#  - 162330 / 250 = 649.32
#  - 164875 / 250 = 659.5
#  - 242923 / 250 = 971.692

______________________ sub-CCTC07_task-rest_eeg ______________________

    - Session 20240325 1414.mff 
    - 2024-03-25 19:25:43.762000+00:00 
    - <RawMff | signal1.bin, 133 x 310630 (310.6 s), ~315.4 MB, data loaded> 
    - time(s): 310.63 
    - first_samp-last_samp: 0 - 310629
    - Duration = 0hr : 05min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5051 / 1000.0 = 5.051
2 - 305139 / 1000.0 = 305.139
Name event1 for sub-CCTC07_task-rest_eeg:_RS
Name event2 for sub-CCTC07_task-rest_eeg:no

______________________ sub-CCTC07_task-taken_eeg ______________________

    - Session 20240325 1431.mff 
    - 2024-03-25 19:31:35.663000+00:00 
    - <RawMff | signal1.bin, 135 x 643354 (643.4 s), ~662.8 MB, data loaded> 
    - time(s): 643.354 
    - first_samp-last_samp: 0 - 643353
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7779 / 1000.0 = 7.779
2 - 310444 / 1000.0 = 310.444
3 - 320596 / 1000.0 = 320.596
4 - 632789 / 1000.0 = 632.789
Name event1 for sub-CCTC07_task-taken_eeg:_ST
Name event2 for sub-CCTC07_task-taken_eeg:no
Name event3 for sub-CCTC07_task-taken_eeg:_IT
Name event4 for sub-CCTC07_task-taken_eeg:no

______________________ sub-CCTC07NETICU_task-sedon1rest_eeg ______________________

    - Session 20240328 1423.mff 
    - 2024-03-28 19:24:00.059000+00:00 
    - <RawMff | signal1.bin, 132 x 1273857 (1273.9 s), ~1.25 GB, data loaded> 
    - time(s): 1273.857 
    - first_samp-last_samp: 0 - 1273856
    - Duration = 0hr : 21min : 13sec
    - Trigger <  ch. 09  >
1 - 5047 / 1000.0 = 5.047
Name event1 for sub-CCTC07NETICU_task-sedon1rest_eeg:sedon1_RS

______________________ sub-CCTC07NETICU_task-sedoffrest_eeg ______________________

    - Session 20240328 1453.mff 
    - 2024-03-28 19:54:13.667000+00:00 
    - <RawMff | signal1.bin, 133 x 610558 (610.6 s), ~619.7 MB, data loaded> 
    - time(s): 610.558 
    - first_samp-last_samp: 0 - 610557
    - Duration = 0hr : 10min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5060 / 1000.0 = 5.06
2 - 605114 / 1000.0 = 605.114
Name event1 for sub-CCTC07NETICU_task-sedoffrest_eeg:sedoff_RS_smin
> smin to replace event1 start time (type "zoom" to plot a closer look):293278
Name event2 for sub-CCTC07NETICU_task-sedoffrest_eeg:no

______________________ sub-CCTC07NETICU_task-sedofftaken_eeg ______________________

    - Session 20240328 1504.mff 
    - 2024-03-28 20:06:55.751000+00:00 
    - <RawMff | signal1.bin, 135 x 642855 (642.9 s), ~662.3 MB, data loaded> 
    - time(s): 642.855 
    - first_samp-last_samp: 0 - 642854
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7358 / 1000.0 = 7.358
2 - 310015 / 1000.0 = 310.015
3 - 320100 / 1000.0 = 320.1
4 - 632293 / 1000.0 = 632.293
Name event1 for sub-CCTC07NETICU_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-CCTC07NETICU_task-sedofftaken_eeg:no
Name event3 for sub-CCTC07NETICU_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-CCTC07NETICU_task-sedofftaken_eeg:no

______________________ sub-CCTC07NETICU_task-sedon1taken_eeg ______________________

    - Session 20240328 1411.mff 
    - 2024-03-28 19:12:29.866000+00:00 
    - <RawMff | signal1.bin, 135 x 643306 (643.3 s), ~662.8 MB, data loaded> 
    - time(s): 643.306 
    - first_samp-last_samp: 0 - 643305
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7686 / 1000.0 = 7.686
2 - 310349 / 1000.0 = 310.349
3 - 320507 / 1000.0 = 320.507
4 - 632702 / 1000.0 = 632.702
Name event1 for sub-CCTC07NETICU_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-CCTC07NETICU_task-sedon1taken_eeg:no
Name event3 for sub-CCTC07NETICU_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-CCTC07NETICU_task-sedon1taken_eeg:no
#
#
#
#
####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

# _____________________________________________________________________________________
# ____________________________________________MSICU05______________________________________________
# ___________ IT02 ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230420 1042 ----> 2023-04-20 15:42:45.953000+00:00 
#     - <RawMff | signal1.bin, 133 x 81532 (326.1 s), ~82.9 MB, data loaded> 
#     - time(s): 326.128 
#     - first_samp-last_samp: 0 - 81531
#     - Duration = 0hr : 05min : 26sec
# (2) Trigger Channels
#     - Trigger ch. 32  
#     - Trigger ch. 42  
#  - 1804 / 250 = 7.216
#  - 79899 / 250 = 319.596
# ___________ RS_ST_Taken ___________
# (1) Data timing... 
#     - Session 20220408 1321 ----> 2022-04-08 18:31:36.824000+00:00 
#     - <RawMff | signal1.bin, 135 x 424286 (1697.1 s), ~437.2 MB, data loaded> 
#     - time(s): 1697.144 
#     - first_samp-last_samp: 0 - 424285
#     - Duration = 0hr : 28min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1553 / 250 = 6.212
#  - 156523 / 250 = 626.092
#  - 244982 / 250 = 979.928
#  - 340549 / 250 = 1362.196
# ___________ rest02 ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230420 1006 ----> 2023-04-20 15:21:53.396000+00:00 
#     - <RawMff | signal1.bin, 133 x 84470 (337.9 s), ~85.9 MB, data loaded> 
#     - time(s): 337.88 
#     - first_samp-last_samp: 0 - 84469
#     - Duration = 0hr : 05min : 37sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#  - 1593 / 250 = 6.372
#  - 84107 / 250 = 336.428
# ___________ ST02 ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230420 1028 ----> 2023-04-20 15:30:54.493000+00:00 
#     - <RawMff | signal1.bin, 133 x 87544 (350.2 s), ~89.0 MB, data loaded> 
#     - time(s): 350.176 
#     - first_samp-last_samp: 0 - 87543
#     - Duration = 0hr : 05min : 50sec
# (2) Trigger Channels
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1812 / 250 = 7.248
#  - 77476 / 250 = 309.904

# ________________________________________________________________________________________________
# ____________________________________________MSICU07______________________________________________
# ___________ RS_ST_Taken_Tennis ___________
# (1) Data timing... 
#     - Session 20220607 1636 ----> 2022-06-07 21:44:25.524000+00:00 
#     - <RawMff | signal1.bin, 135 x 496770 (1987.1 s), ~511.8 MB, data loaded> 
#     - time(s): 1987.08 
#     - first_samp-last_samp: 0 - 496769
#     - Duration = 0hr : 33min : 07sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1558 / 250 = 6.232
#  - 150059 / 250 = 600.236
#  - 250099 / 250 = 1000.396
#  - 427043 / 250 = 1708.172

# ________________________________________________________________________________________________
# ____________________________________________MSICU09______________________________________________
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20220726 1357 ----> 2022-07-26 18:59:23.460000+00:00 
#     - <RawMff | signal1.bin, 133 x 175570 (702.3 s), ~178.3 MB, data loaded> 
#     - time(s): 702.28 
#     - first_samp-last_samp: 0 - 175569
#     - Duration = 0hr : 11min : 42sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 959 / 250 = 3.836
#  - 94125 / 250 = 376.5
# ___________ taken02 ___________
# (1) Data timing... 
#     - Session 20220728 1242 ----> 2022-07-28 17:45:27.760000+00:00 
#     - <RawMff | signal1.bin, 133 x 172419 (689.7 s), ~175.1 MB, data loaded> 
#     - time(s): 689.676 
#     - first_samp-last_samp: 0 - 172418
#     - Duration = 0hr : 11min : 29sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 944 / 250 = 3.776
#  - 90050 / 250 = 360.2
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20220726 1339 ----> 2022-07-26 18:42:54.728000+00:00 
#     - <RawMff | signal1.bin, 132 x 91334 (365.3 s), ~92.2 MB, data loaded> 
#     - time(s): 365.336 
#     - first_samp-last_samp: 0 - 91333
#     - Duration = 0hr : 06min : 05sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1472 / 250 = 5.888
# ___________ rest02 ___________
# (1) Data timing... 
#     - Session 20220728 1226 ----> 2022-07-28 17:29:57.772000+00:00 
#     - <RawMff | signal1.bin, 132 x 90085 (360.3 s), ~90.9 MB, data loaded> 
#     - time(s): 360.34 
#     - first_samp-last_samp: 0 - 90084
#     - Duration = 0hr : 06min : 00sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1495 / 250 = 5.98

# ________________________________________________________________________________________________
# ____________________________________________MSICU10______________________________________________
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20220727 0858 ----> 2022-07-27 14:09:49.536000+00:00 
#     - <RawMff | signal1.bin, 132 x 90123 (360.5 s), ~90.9 MB, data loaded> 
#     - time(s): 360.492 
#     - first_samp-last_samp: 0 - 90122
#     - Duration = 0hr : 06min : 00sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1523 / 250 = 6.092
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20220727 0926 ----> 2022-07-27 14:32:38.148000+00:00 
#     - <RawMff | signal1.bin, 133 x 206319 (825.3 s), ~209.5 MB, data loaded> 
#     - time(s): 825.276 
#     - first_samp-last_samp: 0 - 206318
#     - Duration = 0hr : 13min : 45sec
# (2) Trigger Channels
#     - Trigger ch. 10  
#     - Trigger ch. 12  
#  - 1254 / 250 = 5.016
#  - 8803 / 250 = 35.212
#  - 124083 / 250 = 496.332
______________________ sub-MSICU10_task-taken_eeg ______________________

    - Session 20220727 0926.mff 
    - 2022-07-27 14:32:38.148000+00:00 
    - <RawMff | signal1.bin, 133 x 206319 (825.3 s), ~209.5 MB, data loaded> 
    - time(s): 825.276 
    - first_samp-last_samp: 0 - 206318
    - Duration = 0hr : 13min : 45sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  > again
1 - 1254 / 250.0 = 5.016
2 - 8803 / 250.0 = 35.212
3 - 124083 / 250.0 = 496.332
Name event1 for sub-MSICU10_task-taken_eeg:plot
Name event1 for sub-MSICU10_task-taken_eeg:no
Name event2 for sub-MSICU10_task-taken_eeg:_ST
Name event3 for sub-MSICU10_task-taken_eeg:_IT


# ________________________________________________________________________________________________
# ____________________________________________MSICU14______________________________________________
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20010207 0240 ----> 2001-02-07 07:42:09.316000+00:00 
#     - <RawMff | signal1.bin, 132 x 103796 (415.2 s), ~104.7 MB, data loaded> 
#     - time(s): 415.184 
#     - first_samp-last_samp: 0 - 103795
#     - Duration = 0hr : 06min : 55sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1015 / 250 = 4.06
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20010207 0250 ----> 2001-02-07 07:57:06.704000+00:00 
#     - <RawMff | signal1.bin, 133 x 170032 (680.1 s), ~172.7 MB, data loaded> 
#     - time(s): 680.128 
#     - first_samp-last_samp: 0 - 170031
#     - Duration = 0hr : 11min : 20sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 960 / 250 = 3.84
#  - 87985 / 250 = 351.94

# ________________________________________________________________________________________________
# ____________________________________________MSICU15______________________________________________
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20010219 1732 ----> 2001-02-19 22:34:20.568000+00:00 
#     - <RawMff | signal1.bin, 132 x 99347 (397.4 s), ~100.2 MB, data loaded> 
#     - time(s): 397.388 
#     - first_samp-last_samp: 0 - 99346
#     - Duration = 0hr : 06min : 37sec
# (2) Trigger Channels
#     - Trigger ch. 9   
#  - 1006 / 250 = 4.024
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20010219 1747 ----> 2001-02-19 22:50:53.076000+00:00 
#     - <RawMff | signal1.bin, 133 x 171207 (684.8 s), ~173.9 MB, data loaded> 
#     - time(s): 684.828 
#     - first_samp-last_samp: 0 - 171206
#     - Duration = 0hr : 11min : 24sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 961 / 250 = 3.844
#  - 88643 / 250 = 354.572

# ________________________________________________________________________________________________
# ____________________________________________MSICU16______________________________________________
# ___________ rest ___________
# (1) Data timing... 
#     - Session 20010227 0443 ----> 2001-02-27 09:54:08.540000+00:00 
#     - <RawMff | signal1.bin, 132 x 103585 (414.3 s), ~104.5 MB, data loaded> 
#     - time(s): 414.34 
#     - first_samp-last_samp: 0 - 103584
#     - Duration = 0hr : 06min : 54sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#  - 1426 / 250 = 5.704
# ___________ taken ___________
# (1) Data timing... 
#     - Session 20010227 0510 ----> 2001-02-27 10:12:08.872000+00:00 
#     - <RawMff | signal1.bin, 133 x 171594 (686.4 s), ~174.3 MB, data loaded> 
#     - time(s): 686.376 
#     - first_samp-last_samp: 0 - 171593
#     - Duration = 0hr : 11min : 26sec
# (2) Trigger Channels
#     - Trigger ch. 12  
#     - Trigger ch. 10  
#  - 942 / 250 = 3.768
#  - 90063 / 250 = 360.252

# ________________________________________________________________________________________________
# ____________________________________________MSICU18______________________________________________
# ___________ RS_ST_Taken ___________
# (1) Data timing... 
#     - Session 20221212 1639 ----> 2022-12-12 21:47:13.448000+00:00 
#     - <RawMff | signal1.bin, 137 x 250316 (1001.3 s), ~261.8 MB, data loaded> 
#     - time(s): 1001.264 
#     - first_samp-last_samp: 0 - 250315
#     - Duration = 0hr : 16min : 41sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1379 / 250 = 5.516
#  - 88936 / 250 = 355.744
#  - 91475 / 250 = 365.9
#  - 167095 / 250 = 668.38
#  - 169634 / 250 = 678.536
#  - 247684 / 250 = 990.736

# ________________________________________________________________________________________________
# ____________________________________________MSICU19______________________________________________

# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230131 1717 ----> 2023-01-31 22:24:48.016000+00:00 
#     - <RawMff | signal1.bin, 137 x 244504 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 978.016 
#     - first_samp-last_samp: 0 - 244503
#     - Duration = 0hr : 16min : 18sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1554 / 250 = 6.216
#  - 84074 / 250 = 336.296
#  - 86617 / 250 = 346.468
#  - 162282 / 250 = 649.128
#  - 164830 / 250 = 659.32
#  - 242879 / 250 = 971.516

# ________________________________________________________________________________________________
# ____________________________________________MSICU21______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230205 1031 ----> 2023-02-05 15:33:39.517000+00:00 
#     - <RawMff | signal1.bin, 137 x 244492 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 977.968 
#     - first_samp-last_samp: 0 - 244491
#     - Duration = 0hr : 16min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1529 / 250 = 6.116
#  - 84046 / 250 = 336.184
#  - 86592 / 250 = 346.368
#  - 162256 / 250 = 649.024
#  - 164807 / 250 = 659.228
#  - 242855 / 250 = 971.42

# ________________________________________________________________________________________________
# ____________________________________________MSICU22______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230208 1712 ----> 2023-02-08 22:13:19.416000+00:00 
#     - <RawMff | signal1.bin, 137 x 244492 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 977.968 
#     - first_samp-last_samp: 0 - 244491
#     - Duration = 0hr : 16min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1525 / 250 = 6.1
#  - 84042 / 250 = 336.168
#  - 86588 / 250 = 346.352
#  - 162252 / 250 = 649.008
#  - 164807 / 250 = 659.228
#  - 242855 / 250 = 971.42

# ________________________________________________________________________________________________
# ____________________________________________MSICU26______________________________________________
# ___________ RS_ST_Taken ___________
# (1) Data timing... 
#     - Session 20230406 1651 ----> 2023-04-06 22:00:37.164000+00:00 
#     - <RawMff | signal1.bin, 137 x 244470 (977.9 s), ~255.7 MB, data loaded> 
#     - time(s): 977.88 
#     - first_samp-last_samp: 0 - 244469
#     - Duration = 0hr : 16min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1521 / 250 = 6.084
#  - 84038 / 250 = 336.152
#  - 86583 / 250 = 346.332
#  - 162247 / 250 = 648.988
#  - 164796 / 250 = 659.184
#  - 242844 / 250 = 971.376

# ________________________________________________________________________________________________
# ____________________________________________MSICU28______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230515 1622 ----> 2023-05-15 21:22:37.007000+00:00 
#     - <RawMff | signal1.bin, 137 x 244495 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 977.98 
#     - first_samp-last_samp: 0 - 244494
#     - Duration = 0hr : 16min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 59  
#     - Trigger ch. 69  
#     - Trigger ch. 62  
#     - Trigger ch. 72  
#     - Trigger ch. 60  
#     - Trigger ch. 70  
#  - 1508 / 250 = 6.032
#  - 84027 / 250 = 336.108
#  - 86603 / 250 = 346.412
#  - 162270 / 250 = 649.08
#  - 164812 / 250 = 659.248
#  - 242862 / 250 = 971.448

# ________________________________________________________________________________________________
# ____________________________________________MSICU29______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230609 1229 ----> 2023-06-09 17:56:09.966000+00:00 
#     - <RawMff | signal1.bin, 137 x 244508 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 978.032 
#     - first_samp-last_samp: 0 - 244507
#     - Duration = 0hr : 16min : 18sec
# (2) Trigger Channels
#     - Trigger ch. 59  
#     - Trigger ch. 69  
#     - Trigger ch. 62  
#     - Trigger ch. 72  
#     - Trigger ch. 60  
#     - Trigger ch. 70  
#  - 1552 / 250 = 6.208
#  - 84074 / 250 = 336.296
#  - 86615 / 250 = 346.46
#  - 162281 / 250 = 649.124
#  - 164826 / 250 = 659.304
#  - 242875 / 250 = 971.5

# ________________________________________________________________________________________________
# ____________________________________________MSICU30______________________________________________
# ___________ RS_ST_Taken ___________
#     > Data was downsampled to 250Hz from 1000.0
# (1) Data timing... 
#     - Session 20230614 1437 ----> 2023-06-14 19:52:12.138000+00:00 
#     - <RawMff | signal1.bin, 137 x 244495 (978.0 s), ~255.7 MB, data loaded> 
#     - time(s): 977.98 
#     - first_samp-last_samp: 0 - 244494
#     - Duration = 0hr : 16min : 17sec
# (2) Trigger Channels
#     - Trigger ch. 09  
#     - Trigger ch. 19  
#     - Trigger ch. 12  
#     - Trigger ch. 22  
#     - Trigger ch. 10  
#     - Trigger ch. 20  
#  - 1520 / 250 = 6.08
#  - 84037 / 250 = 336.148
#  - 86604 / 250 = 346.416
#  - 162268 / 250 = 649.072
#  - 164815 / 250 = 659.26
#  - 242864 / 250 = 971.456

______________________ sub-MSICU32_task-rest_eeg ______________________

    - Session 20240220 2041.mff 
    - 2024-02-21 01:44:07.407000+00:00 
    - <RawMff | signal1.bin, 131 x 362672 (362.7 s), ~362.7 MB, data loaded> 
    - time(s): 362.672 
    - first_samp-last_samp: 0 - 362671
    - Duration = 0hr : 06min : 02sec
Name file for sub-MSICU32_task-rest_eeg:_RS
> smin value (samplingrate=1000.0) for sub-MSICU32_task-rest_eeg:10000

______________________ sub-MSICU32_task-taken_eeg ______________________

    - Session 20240220 2022.mff 
    - 2024-02-21 01:30:46.400000+00:00 
    - <RawMff | signal1.bin, 135 x 160839 (643.4 s), ~165.8 MB, data loaded> 
    - time(s): 643.356 
    - first_samp-last_samp: 0 - 160838
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 1937 / 250.0 = 7.748
2 - 77606 / 250.0 = 310.424
3 - 80146 / 250.0 = 320.584
4 - 158193 / 250.0 = 632.772
Name event1 for sub-MSICU32_task-taken_eeg:_ST
Name event2 for sub-MSICU32_task-taken_eeg:no
Name event3 for sub-MSICU32_task-taken_eeg:_IT
Name event4 for sub-MSICU32_task-taken_eeg:no

______________________ sub-MSICU38_task-taken_eeg ______________________

    - Session 20240311 1457.mff 
    - 2024-03-11 19:58:58.964000+00:00 
    - <RawMff | signal1.bin, 135 x 643203 (643.2 s), ~662.7 MB, data loaded> 
    - time(s): 643.203 
    - first_samp-last_samp: 0 - 643202
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7594 / 1000.0 = 7.594
2 - 310252 / 1000.0 = 310.252
3 - 320405 / 1000.0 = 320.405
4 - 632603 / 1000.0 = 632.603
Name event1 for sub-MSICU38_task-taken_eeg:_ST
Name event2 for sub-MSICU38_task-taken_eeg:no
Name event3 for sub-MSICU38_task-taken_eeg:_IT
Name event4 for sub-MSICU38_task-taken_eeg:no

______________________ sub-MSICU38_task-rest_eeg ______________________

    - Session 20240311 1443.mff 
    - 2024-03-11 19:52:20.669000+00:00 
    - <RawMff | signal1.bin, 134 x 310579 (310.6 s), ~317.7 MB, data loaded> 
    - time(s): 310.579 
    - first_samp-last_samp: 0 - 310578
    - Duration = 0hr : 05min : 10sec
    - Trigger <  ch. cal*>
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5048 / 1000.0 = 5.048
2 - 5149 / 1000.0 = 5.149
3 - 10749 / 1000.0 = 10.749
4 - 16200 / 1000.0 = 16.2
5 - 305115 / 1000.0 = 305.115
Name event1 for sub-MSICU38_task-rest_eeg:no
Name event2 for sub-MSICU38_task-rest_eeg:_RS
Name event3 for sub-MSICU38_task-rest_eeg:no
Name event4 for sub-MSICU38_task-rest_eeg:no
Name event5 for sub-MSICU38_task-rest_eeg:no

______________________ sub-MSICU42_task-rest_eeg ______________________

    - Session 20240417 1431.mff 
    - 2024-04-17 19:33:32.460000+00:00 
    - <RawMff | signal1.bin, 133 x 90207 (360.8 s), ~91.7 MB, data loaded> 
    - time(s): 360.828 
    - first_samp-last_samp: 0 - 90206
    - Duration = 0hr : 06min : 00sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 1264 / 250.0 = 5.056
2 - 88839 / 250.0 = 355.356
Name event1 for sub-MSICU42_task-rest_eeg:_RS
Name event2 for sub-MSICU42_task-rest_eeg:no

______________________ sub-MSICU42_task-taken_eeg ______________________

    - Session 20240417 1442.mff 
    - 2024-04-17 19:44:33.396000+00:00 
    - <RawMff | signal1.bin, 135 x 160889 (643.6 s), ~165.9 MB, data loaded> 
    - time(s): 643.556 
    - first_samp-last_samp: 0 - 160888
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 1984 / 250.0 = 7.936
2 - 77651 / 250.0 = 310.604
3 - 80189 / 250.0 = 320.756
4 - 158238 / 250.0 = 632.952
Name event1 for sub-MSICU42_task-taken_eeg:_ST
Name event2 for sub-MSICU42_task-taken_eeg:no
Name event3 for sub-MSICU42_task-taken_eeg:_IT
Name event4 for sub-MSICU42_task-taken_eeg:no

______________________ sub-MSICU43_task-rest_eeg ______________________


    - Session 20240417 1005.mff 
    - 2024-04-17 15:13:20.578000+00:00 
    - <RawMff | signal1.bin, 133 x 360475 (360.5 s), ~366.0 MB, data loaded> 
    - time(s): 360.475 
    - first_samp-last_samp: 0 - 360474
    - Duration = 0hr : 06min : 00sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5070 / 1000.0 = 5.07
2 - 355144 / 1000.0 = 355.144
Name event1 for sub-MSICU43_task-rest_eeg:_RS
Name event2 for sub-MSICU43_task-rest_eeg:no

______________________ sub-MSICU43_task-taken_eeg ______________________

    - Session 20240417 1019.mff 
    - 2024-04-17 15:21:21.902000+00:00 
    - <RawMff | signal1.bin, 135 x 643505 (643.5 s), ~663.0 MB, data loaded> 
    - time(s): 643.505 
    - first_samp-last_samp: 0 - 643504
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7740 / 1000.0 = 7.74
2 - 310591 / 1000.0 = 310.591
3 - 320742 / 1000.0 = 320.742
4 - 632936 / 1000.0 = 632.936
Name event1 for sub-MSICU43_task-taken_eeg:_ST
Name event2 for sub-MSICU43_task-taken_eeg:no
Name event3 for sub-MSICU43_task-taken_eeg:_IT
Name event4 for sub-MSICU43_task-taken_eeg:no

______________________ sub-MSICU47_task-rest_eeg ______________________

    - Session 20240423 1659.mff 
    - 2024-04-23 22:07:22.452000+00:00 
    - <RawMff | signal1.bin, 133 x 539157 (539.2 s), ~547.3 MB, data loaded> 
    - time(s): 539.157 
    - first_samp-last_samp: 0 - 539156
    - Duration = 0hr : 08min : 59sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5254 / 1000.0 = 5.254
2 - 355316 / 1000.0 = 355.316
Name event1 for sub-MSICU47_task-rest_eeg:_RS
Name event2 for sub-MSICU47_task-rest_eeg:no

______________________ sub-MSICU47_task-taken_eeg ______________________

    - Session 20240423 1716.mff 
    - 2024-04-23 22:18:15.888000+00:00 
    - <RawMff | signal1.bin, 135 x 643853 (643.9 s), ~663.3 MB, data loaded> 
    - time(s): 643.853 
    - first_samp-last_samp: 0 - 643852
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8251 / 1000.0 = 8.251
2 - 310924 / 1000.0 = 310.924
3 - 321074 / 1000.0 = 321.074
4 - 633272 / 1000.0 = 633.272
Name event1 for sub-MSICU47_task-taken_eeg:_ST
Name event2 for sub-MSICU47_task-taken_eeg:no
Name event3 for sub-MSICU47_task-taken_eeg:_IT
Name event4 for sub-MSICU47_task-taken_eeg:np











####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################

####
3
3
3

3

______________________ sub-NETICU01_task-sedon1taken_eeg ______________________

    - Session 20200304 1111.mff 
    - 2020-03-04 16:18:51.748000+00:00 
    - <RawMff | signal1.bin, 133 x 190730 (762.9 s), ~193.7 MB, data loaded> 
    - time(s): 762.92 
    - first_samp-last_samp: 0 - 190729
    - Duration = 0hr : 12min : 42sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2670 / 250.0 = 10.68
2 - 104505 / 250.0 = 418.02
Name event1 for sub-NETICU01_task-sedon1taken_eeg:plot
Name event1 for sub-NETICU01_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU01_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU01_task-sedofftaken_eeg ______________________

    - Session 20200304 1229.mff 
    - 2020-03-04 17:34:36.256000+00:00 
    - <RawMff | signal1.bin, 133 x 175181 (700.7 s), ~177.9 MB, data loaded> 
    - time(s): 700.724 
    - first_samp-last_samp: 0 - 175180
    - Duration = 0hr : 11min : 40sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2043 / 250.0 = 8.172
2 - 84319 / 250.0 = 337.276
Name event1 for sub-NETICU01_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU01_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU01_task-sedon1rest_eeg ______________________

    - Session 20200304 1047.mff 
    - 2020-03-04 15:54:29.144000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU01_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=250.0) for sub-NETICU01_task-sedon1rest_eeg:25000

______________________ sub-NETICU01_task-sedoffrest_eeg ______________________

    - Session 20200304 1216.mff 
    - 2020-03-04 17:18:12.316000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU01_task-sedoffrest_eeg:plot
Name for file sub-NETICU01_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=250.0) for sub-NETICU01_task-sedoffrest_eeg:69500

______________________ sub-NETICU02_task-sedoffrest_eeg ______________________

    - Session 20200306 1030.mff 
    - 2020-03-06 15:33:54.764000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU02_task-sedoffrest_eeg:plot
Name for file sub-NETICU02_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=250.0) for sub-NETICU02_task-sedoffrest_eeg:3750

______________________ sub-NETICU02_task-sedon1taken_eeg ______________________

    - Session 20200306 0935.mff 
    - 2020-03-06 14:41:16.368000+00:00 
    - <RawMff | signal1.bin, 133 x 185144 (740.6 s), ~188.0 MB, data loaded> 
    - time(s): 740.576 
    - first_samp-last_samp: 0 - 185143
    - Duration = 0hr : 12min : 20sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2778 / 250.0 = 11.112
2 - 103678 / 250.0 = 414.712
Name event1 for sub-NETICU02_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU02_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU02_task-sedon1rest_eeg ______________________

    - Session 20200306 0914.mff 
    - 2020-03-06 14:24:38.580000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU02_task-sedon1rest_eeg:plot
Name for file sub-NETICU02_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=250.0) for sub-NETICU02_task-sedon1rest_eeg:50000

______________________ sub-NETICU02_task-sedofftaken_eeg ______________________

    - Session 20200306 1012.mff 
    - 2020-03-06 15:18:10.308000+00:00 
    - <RawMff | signal1.bin, 133 x 173268 (693.1 s), ~176.0 MB, data loaded> 
    - time(s): 693.072 
    - first_samp-last_samp: 0 - 173267
    - Duration = 0hr : 11min : 33sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
1 - 2081 / 250.0 = 8.324
2 - 90725 / 250.0 = 362.9
Name event1 for sub-NETICU02_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU02_task-sedofftaken_eeg:sedoff_IT
Name event2 for sub-NETICU02_task-sedofftaken_eeg:sedoff_ST

______________________ sub-NETICU03_task-sedon1rest_eeg ______________________

    - Session 20200306 1804.mff 
    - 2020-03-06 23:33:02.168000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU03_task-sedon1rest_eeg:plot
Name for file sub-NETICU03_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=250.0) for sub-NETICU03_task-sedon1rest_eeg:12500

______________________ sub-NETICU03_task-sedofftaken_eeg ______________________

    - Session 20200306 1914.mff 
    - 2020-03-07 00:21:29.552000+00:00 
    - <RawMff | signal1.bin, 133 x 178931 (715.7 s), ~181.7 MB, data loaded> 
    - time(s): 715.724 
    - first_samp-last_samp: 0 - 178930
    - Duration = 0hr : 11min : 55sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2040 / 250.0 = 8.16
2 - 88278 / 250.0 = 353.112
Name event1 for sub-NETICU03_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU03_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU03_task-sedon1taken_eeg ______________________

    - Session 20200306 1844.mff 
    - 2020-03-06 23:46:16.668000+00:00 
    - <RawMff | signal1.bin, 133 x 188356 (753.4 s), ~191.3 MB, data loaded> 
    - time(s): 753.424 
    - first_samp-last_samp: 0 - 188355
    - Duration = 0hr : 12min : 33sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 2675 / 250.0 = 10.7
2 - 97108 / 250.0 = 388.432
Name event1 for sub-NETICU03_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU03_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU03_task-sedoffrest_eeg ______________________

    - Session 20200306 1933.mff 
    - 2020-03-07 00:35:02.716000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU03_task-sedoffrest_eeg:plot
Name for file sub-NETICU03_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=250.0) for sub-NETICU03_task-sedoffrest_eeg:69500

______________________ sub-NETICU04_task-sedon1taken_eeg ______________________

    - Session 20371028 1019.mff 
    - 2037-10-28 15:30:24.148000+00:00 
    - <RawMff | signal1.bin, 133 x 174993 (700.0 s), ~177.7 MB, data loaded> 
    - time(s): 699.972 
    - first_samp-last_samp: 0 - 174992
    - Duration = 0hr : 11min : 39sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1365 / 250.0 = 5.46
2 - 90339 / 250.0 = 361.356
Name event1 for sub-NETICU04_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU04_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU04_task-sedoffrest_eeg ______________________

    - Session 20371028 1133.mff 
    - 2037-10-28 16:36:34.650000+00:00 
    - <RawMff | signal1.bin, 131 x 601879 (601.9 s), ~601.7 MB, data loaded> 
    - time(s): 601.879 
    - first_samp-last_samp: 0 - 601878
    - Duration = 0hr : 10min : 01sec
Name file for sub-NETICU04_task-sedoffrest_eeg:plot
Name for file sub-NETICU04_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU04_task-sedoffrest_eeg:278000

______________________ sub-NETICU04_task-sedofftaken_eeg ______________________

    - Session 20371028 1116.mff 
    - 2037-10-28 16:21:01.752000+00:00 
    - <RawMff | signal1.bin, 133 x 180256 (721.0 s), ~183.1 MB, data loaded> 
    - time(s): 721.024 
    - first_samp-last_samp: 0 - 180255
    - Duration = 0hr : 12min : 01sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 929 / 250.0 = 3.716
2 - 95972 / 250.0 = 383.888

Name event1 for sub-NETICU04_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU04_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU04_task-sedon1rest_eeg ______________________

    - Session 20371028 0957.mff 
    - 2037-10-28 15:08:08.708000+00:00 
    - <RawMff | signal1.bin, 131 x 604881 (604.9 s), ~604.7 MB, data loaded> 
    - time(s): 604.881 
    - first_samp-last_samp: 0 - 604880
    - Duration = 0hr : 10min : 04sec
Name file for sub-NETICU04_task-sedon1rest_eeg:plot
Name for file sub-NETICU04_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU04_task-sedon1rest_eeg:278000

______________________ sub-NETICU05_task-sedoffrest_eeg ______________________

    - Session 20210217 1130.mff 
    - 2021-02-17 16:32:26.357000+00:00 
    - <RawMff | signal1.bin, 131 x 610578 (610.6 s), ~610.4 MB, data loaded> 
    - time(s): 610.578 
    - first_samp-last_samp: 0 - 610577
    - Duration = 0hr : 10min : 10sec
Name file for sub-NETICU05_task-sedoffrest_eeg:plot
Name for file sub-NETICU05_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU05_task-sedoffrest_eeg:278000

______________________ sub-NETICU05_task-sedon1taken_eeg ______________________

    - Session 20210217 1037.mff 
    - 2021-02-17 15:43:32.004000+00:00 
    - <RawMff | signal1.bin, 133 x 306437 (1225.7 s), ~311.1 MB, data loaded> 
    - time(s): 1225.748 
    - first_samp-last_samp: 0 - 306436
    - Duration = 0hr : 20min : 25sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1557 / 250.0 = 6.228
2 - 228342 / 250.0 = 913.368
Name event1 for sub-NETICU05_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU05_task-sedon1taken_eeg:plot
Name event2 for sub-NETICU05_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU05_task-sedon1rest_eeg ______________________

    - Session 20210217 1014.mff 
    - 2021-02-17 15:26:23.850000+00:00 
    - <RawMff | signal1.bin, 131 x 610678 (610.7 s), ~610.5 MB, data loaded> 
    - time(s): 610.678 
    - first_samp-last_samp: 0 - 610677
    - Duration = 0hr : 10min : 10sec
Name file for sub-NETICU05_task-sedon1rest_eeg:plot
Name for file sub-NETICU05_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU05_task-sedon1rest_eeg:278000

______________________ sub-NETICU05_task-sedofftaken_eeg ______________________

    - Session 20210217 1111.mff 
    - 2021-02-17 16:18:30.356000+00:00 
    - <RawMff | signal1.bin, 133 x 161369 (645.5 s), ~163.9 MB, data loaded> 
    - time(s): 645.476 
    - first_samp-last_samp: 0 - 161368
    - Duration = 0hr : 10min : 45sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 923 / 250.0 = 3.692
2 - 81736 / 250.0 = 326.944
Name event1 for sub-NETICU05_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU05_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU06_task-sedon1taken_eeg ______________________

    - Session 20210311 2137.mff 
    - 2021-03-12 02:40:27.676000+00:00 
    - <RawMff | signal1.bin, 133 x 180781 (723.1 s), ~183.6 MB, data loaded> 
    - time(s): 723.124 
    - first_samp-last_samp: 0 - 180780
    - Duration = 0hr : 12min : 03sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1570 / 250.0 = 6.28
2 - 98634 / 250.0 = 394.536
Name event1 for sub-NETICU06_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU06_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU06_task-sedoffrest_eeg ______________________

    - Session 20210311 2213.mff 
    - 2021-03-12 03:15:50.670000+00:00 
    - <RawMff | signal1.bin, 131 x 601380 (601.4 s), ~601.2 MB, data loaded> 
    - time(s): 601.38 
    - first_samp-last_samp: 0 - 601379
    - Duration = 0hr : 10min : 01sec
Name file for sub-NETICU06_task-sedoffrest_eeg:plot
Name for file sub-NETICU06_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU06_task-sedoffrest_eeg:278000

______________________ sub-NETICU06_task-sedofftaken_eeg ______________________

    - Session 20210311 2226.mff 
    - 2021-03-12 03:28:34.472000+00:00 
    - <RawMff | signal1.bin, 133 x 172581 (690.3 s), ~175.3 MB, data loaded> 
    - time(s): 690.324 
    - first_samp-last_samp: 0 - 172580
    - Duration = 0hr : 11min : 30sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 927 / 250.0 = 3.708
2 - 88923 / 250.0 = 355.692
Name event1 for sub-NETICU06_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU06_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU06_task-sedon1rest_eeg ______________________

    - Session 20210311 2110.mff 
    - 2021-03-12 02:26:06.565000+00:00 
    - <RawMff | signal1.bin, 131 x 612430 (612.4 s), ~612.3 MB, data loaded> 
    - time(s): 612.43 
    - first_samp-last_samp: 0 - 612429
    - Duration = 0hr : 10min : 12sec
Name file for sub-NETICU06_task-sedon1rest_eeg:plot
Name for file sub-NETICU06_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU06_task-sedon1rest_eeg:210000

______________________ sub-NETICU08_task-sedon2rest_eeg ______________________

    - Session 20211001 1545.mff 
    - 2021-10-01 20:51:53.849000+00:00 
    - <RawMff | signal1.bin, 131 x 601628 (601.6 s), ~601.5 MB, data loaded> 
    - time(s): 601.628 
    - first_samp-last_samp: 0 - 601627
    - Duration = 0hr : 10min : 01sec
Name file for sub-NETICU08_task-sedon2rest_eeg:plot
Name for file sub-NETICU08_task-sedon2rest_eeg:sedon2_RS
> smin value (samplingrate=1000.0) for sub-NETICU08_task-sedon2rest_eeg:278000

______________________ sub-NETICU08_task-sedofftaken_eeg ______________________

    - Session 20211001 1518.mff 
    - 2021-10-01 20:28:05.460000+00:00 
    - <RawMff | signal1.bin, 134 x 172007 (688.0 s), ~176.0 MB, data loaded> 
    - time(s): 688.028 
    - first_samp-last_samp: 0 - 172006
    - Duration = 0hr : 11min : 28sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
    - Trigger <  ch. 15  >
1 - 1808 / 250.0 = 7.232
2 - 83277 / 250.0 = 333.108
3 - 168965 / 250.0 = 675.86
Name event1 for sub-NETICU08_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU08_task-sedofftaken_eeg:sedoff_IT
Name event2 for sub-NETICU08_task-sedofftaken_eeg:sedoff_ST
Name event3 for sub-NETICU08_task-sedofftaken_eeg:no

______________________ sub-NETICU08_task-sedon2taken_eeg ______________________

    - Session 20211001 1602.mff 
    - 2021-10-01 21:03:42.724000+00:00 
    - <RawMff | signal1.bin, 134 x 169005 (676.0 s), ~173.0 MB, data loaded> 
    - time(s): 676.02 
    - first_samp-last_samp: 0 - 169004
    - Duration = 0hr : 11min : 16sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 15  >
1 - 918 / 250.0 = 3.672
2 - 81453 / 250.0 = 325.812
3 - 164497 / 250.0 = 657.988
Name event1 for sub-NETICU08_task-sedon2taken_eeg:plot
Name event1 for sub-NETICU08_task-sedon2taken_eeg:sedon2_ST
Name event2 for sub-NETICU08_task-sedon2taken_eeg:sedon2_IT
Name event3 for sub-NETICU08_task-sedon2taken_eeg:no

______________________ sub-NETICU11_task-sedon1taken_eeg ______________________

    - Session 20211116 1059.mff 
    - 2021-11-16 16:03:48.024000+00:00 
    - <RawMff | signal1.bin, 134 x 176456 (705.8 s), ~180.6 MB, data loaded> 
    - time(s): 705.824 
    - first_samp-last_samp: 0 - 176455
    - Duration = 0hr : 11min : 45sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 1673 / 250.0 = 6.692
2 - 84674 / 250.0 = 338.696
3 - 170077 / 250.0 = 680.308
Name event1 for sub-NETICU11_task-sedon1taken_eeg:exit

______________________ sub-NETICU11_task-sedon1taken_eeg ______________________

    - Session 20211116 1059.mff 
    - 2021-11-16 16:03:48.024000+00:00 
    - <RawMff | signal1.bin, 134 x 176456 (705.8 s), ~180.6 MB, data loaded> 
    - time(s): 705.824 
    - first_samp-last_samp: 0 - 176455
    - Duration = 0hr : 11min : 45sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 1673 / 250.0 = 6.692
2 - 84674 / 250.0 = 338.696
3 - 170077 / 250.0 = 680.308
Name event1 for sub-NETICU11_task-sedon1taken_eeg:plot
Name event1 for sub-NETICU11_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU11_task-sedon1taken_eeg:sedon1_IT
Name event3 for sub-NETICU11_task-sedon1taken_eeg:no

______________________ sub-NETICU11_task-sedoffrest_eeg ______________________

    - Session 20211116 1118.mff 
    - 2021-11-16 16:29:13.595000+00:00 
    - <RawMff | signal1.bin, 131 x 613777 (613.8 s), ~613.6 MB, data loaded> 
    - time(s): 613.777 
    - first_samp-last_samp: 0 - 613776
    - Duration = 0hr : 10min : 13sec
Name file for sub-NETICU11_task-sedoffrest_eeg:plot
Name for file sub-NETICU11_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU11_task-sedoffrest_eeg:278000

______________________ sub-NETICU11_task-sedon1rest_eeg ______________________

    - Session 20211116 1042.mff 
    - 2021-11-16 15:48:08.411000+00:00 
    - <RawMff | signal1.bin, 131 x 602581 (602.6 s), ~602.4 MB, data loaded> 
    - time(s): 602.581 
    - first_samp-last_samp: 0 - 602580
    - Duration = 0hr : 10min : 02sec
Name file for sub-NETICU11_task-sedon1rest_eeg:plot
Name for file sub-NETICU11_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU11_task-sedon1rest_eeg:278000

______________________ sub-NETICU11_task-sedofftaken_eeg ______________________

    - Session 20211116 1140.mff 
    - 2021-11-16 16:42:56.520000+00:00 
    - <RawMff | signal1.bin, 134 x 170118 (680.5 s), ~174.1 MB, data loaded> 
    - time(s): 680.472 
    - first_samp-last_samp: 0 - 170117
    - Duration = 0hr : 11min : 20sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 1005 / 250.0 = 4.02
2 - 81220 / 250.0 = 324.88
3 - 164337 / 250.0 = 657.348
Name event1 for sub-NETICU11_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU11_task-sedofftaken_eeg:sedoff_IT
Name event3 for sub-NETICU11_task-sedofftaken_eeg:no

______________________ sub-NETICU13_task-sedon1taken_eeg ______________________

    - Session 20220329 0715.mff 
    - 2022-03-29 12:21:29.460000+00:00 
    - <RawMff | signal1.bin, 134 x 178032 (712.1 s), ~182.2 MB, data loaded> 
    - time(s): 712.128 
    - first_samp-last_samp: 0 - 178031
    - Duration = 0hr : 11min : 52sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 1477 / 250.0 = 5.908
2 - 83968 / 250.0 = 335.872
3 - 173565 / 250.0 = 694.26
Name event1 for sub-NETICU13_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU13_task-sedon1taken_eeg:sedon1_IT
Name event3 for sub-NETICU13_task-sedon1taken_eeg:no

______________________ sub-NETICU13_task-sedon1rest_eeg ______________________

    - Session 20220329 0640.mff 
    - 2022-03-29 12:03:51.583000+00:00 
    - <RawMff | signal1.bin, 131 x 601779 (601.8 s), ~601.6 MB, data loaded> 
    - time(s): 601.779 
    - first_samp-last_samp: 0 - 601778
    - Duration = 0hr : 10min : 01sec
Name file for sub-NETICU13_task-sedon1rest_eeg:plot
Name for file sub-NETICU13_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU13_task-sedon1rest_eeg:278000

______________________ sub-NETICU13_task-sedofftaken_eeg ______________________

    - Session 20220329 0800.mff 
    - 2022-03-29 13:03:34.360000+00:00 
    - <RawMff | signal1.bin, 134 x 181831 (727.3 s), ~186.1 MB, data loaded> 
    - time(s): 727.324 
    - first_samp-last_samp: 0 - 181830
    - Duration = 0hr : 12min : 07sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 928 / 250.0 = 3.712
2 - 86130 / 250.0 = 344.52
3 - 179011 / 250.0 = 716.044
Name event1 for sub-NETICU13_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU13_task-sedofftaken_eeg:sedoff_IT
Name event3 for sub-NETICU13_task-sedofftaken_eeg:no

______________________ sub-NETICU13_task-sedoffrest_eeg ______________________

    - Session 20220329 0734.mff 
    - 2022-03-29 12:48:11.424000+00:00 
    - <RawMff | signal1.bin, 131 x 631678 (631.7 s), ~631.5 MB, data loaded> 
    - time(s): 631.678 
    - first_samp-last_samp: 0 - 631677
    - Duration = 0hr : 10min : 31sec
Name file for sub-NETICU13_task-sedoffrest_eeg:plot
Name for file sub-NETICU13_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU13_task-sedoffrest_eeg:5000

______________________ sub-NETICU15_task-sedon1rest_eeg ______________________

    - Session 20220526 1730.mff 
    - 2022-05-26 22:32:00.072000+00:00 
    - <RawMff | signal1.bin, 131 x 568979 (569.0 s), ~568.8 MB, data loaded> 
    - time(s): 568.979 
    - first_samp-last_samp: 0 - 568978
    - Duration = 0hr : 09min : 28sec
Name file for sub-NETICU15_task-sedon1rest_eeg:plot
Name for file sub-NETICU15_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU15_task-sedon1rest_eeg:5000

______________________ sub-NETICU15_task-sedon1taken_eeg ______________________

    - Session 20220526 1701.mff 
    - 2022-05-26 22:08:45.408000+00:00 
    - <RawMff | signal1.bin, 134 x 185544 (742.2 s), ~189.9 MB, data loaded> 
    - time(s): 742.176 
    - first_samp-last_samp: 0 - 185543
    - Duration = 0hr : 12min : 22sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 1567 / 250.0 = 6.268
2 - 92911 / 250.0 = 371.644
3 - 182087 / 250.0 = 728.348
Name event1 for sub-NETICU15_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU15_task-sedon1taken_eeg:plot
Name event2 for sub-NETICU15_task-sedon1taken_eeg:sedon1_IT
Name event3 for sub-NETICU15_task-sedon1taken_eeg:no

______________________ sub-NETICU15_task-sedofftaken_eeg ______________________

    - Session 20220526 1808.mff 
    - 2022-05-26 23:10:48.996000+00:00 
    - <RawMff | signal1.bin, 134 x 185106 (740.4 s), ~189.4 MB, data loaded> 
    - time(s): 740.424 
    - first_samp-last_samp: 0 - 185105
    - Duration = 0hr : 12min : 20sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 13  >
1 - 933 / 250.0 = 3.732
2 - 82592 / 250.0 = 330.368
3 - 181464 / 250.0 = 725.856
Name event1 for sub-NETICU15_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU15_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU15_task-sedofftaken_eeg:sedoff_IT
Name event3 for sub-NETICU15_task-sedofftaken_eeg:no

______________________ sub-NETICU15_task-sedoffrest_eeg ______________________

    - Session 20220526 1744.mff 
    - 2022-05-26 22:51:41.952000+00:00 
    - <RawMff | signal1.bin, 131 x 901470 (901.5 s), ~901.2 MB, data loaded> 
    - time(s): 901.47 
    - first_samp-last_samp: 0 - 901469
    - Duration = 0hr : 15min : 01sec
Name file for sub-NETICU15_task-sedoffrest_eeg:plot
Name for file sub-NETICU15_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU15_task-sedoffrest_eeg:573000

______________________ sub-NETICU17_task-sedofftaken_eeg ______________________

    - Session 20220624 1231.mff 
    - 2022-06-24 17:34:22.164000+00:00 
    - <RawMff | signal1.bin, 133 x 165132 (660.5 s), ~167.7 MB, data loaded> 
    - time(s): 660.528 
    - first_samp-last_samp: 0 - 165131
    - Duration = 0hr : 11min : 00sec
    - Trigger <  ch. 10  >
    - Trigger <  ch. 12  >
1 - 1086 / 250.0 = 4.344
2 - 83420 / 250.0 = 333.68
Name event1 for sub-NETICU17_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU17_task-sedofftaken_eeg:sedoff_IT
Name event2 for sub-NETICU17_task-sedofftaken_eeg:sedoff_ST

______________________ sub-NETICU17_task-sedon1rest_eeg ______________________


    - Session 20220624 1116.mff 
    - 2022-06-24 16:25:51.347000+00:00 
    - <RawMff | signal1.bin, 131 x 737275 (737.3 s), ~737.0 MB, data loaded> 
    - time(s): 737.275 
    - first_samp-last_samp: 0 - 737274
    - Duration = 0hr : 12min : 17sec
Name file for sub-NETICU17_task-sedon1rest_eeg:plot
Name for file sub-NETICU17_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU17_task-sedon1rest_eeg:278000

______________________ sub-NETICU17_task-sedoffrest_eeg ______________________

    - Session 20220624 1218.mff 
    - 2022-06-24 17:19:25.612000+00:00 
    - <RawMff | signal1.bin, 131 x 639478 (639.5 s), ~639.3 MB, data loaded> 
    - time(s): 639.478 
    - first_samp-last_samp: 0 - 639477
    - Duration = 0hr : 10min : 39sec
Name file for sub-NETICU17_task-sedoffrest_eeg:plot
Name for file sub-NETICU17_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU17_task-sedoffrest_eeg:250000

______________________ sub-NETICU17_task-sedon1taken_eeg ______________________

    - Session 20220624 1139.mff 
    - 2022-06-24 16:43:49.468000+00:00 
    - <RawMff | signal1.bin, 134 x 196406 (785.6 s), ~201.0 MB, data loaded> 
    - time(s): 785.624 
    - first_samp-last_samp: 0 - 196405
    - Duration = 0hr : 13min : 05sec
    - Trigger <  ch. 21  >
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1639 / 250.0 = 6.556
2 - 7749 / 250.0 = 30.996
3 - 103503 / 250.0 = 414.012
Name event1 for sub-NETICU17_task-sedon1taken_eeg:plot
Name event1 for sub-NETICU17_task-sedon1taken_eeg:no
Name event2 for sub-NETICU17_task-sedon1taken_eeg:sedon1_ST
Name event3 for sub-NETICU17_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU18_task-sedon1taken_eeg ______________________

    - Session 20010123 1936.mff 
    - 2001-01-24 00:39:50.516000+00:00 
    - <RawMff | signal1.bin, 133 x 175182 (700.7 s), ~177.9 MB, data loaded> 
    - time(s): 700.728 
    - first_samp-last_samp: 0 - 175181
    - Duration = 0hr : 11min : 40sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1542 / 250.0 = 6.168
2 - 93548 / 250.0 = 374.192
Name event1 for sub-NETICU18_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU18_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU18_task-sedoffrest_eeg ______________________

    - Session 20010123 2010.mff 
    - 2001-01-24 01:12:57.519000+00:00 
    - <RawMff | signal1.bin, 131 x 1260557 (1260.6 s), ~1.23 GB, data loaded> 
    - time(s): 1260.557 
    - first_samp-last_samp: 0 - 1260556
    - Duration = 0hr : 21min : 00sec
Name file for sub-NETICU18_task-sedoffrest_eeg:plot
Name for file sub-NETICU18_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU18_task-sedoffrest_eeg:650000

______________________ sub-NETICU18_task-sedon1rest_eeg ______________________

    - Session 20010123 1953.mff 
    - 2001-01-24 00:58:32.258000+00:00 
    - <RawMff | signal1.bin, 131 x 600480 (600.5 s), ~600.3 MB, data loaded> 
    - time(s): 600.48 
    - first_samp-last_samp: 0 - 600479
    - Duration = 0hr : 10min : 00sec
Name file for sub-NETICU18_task-sedon1rest_eeg:plot
Name for file sub-NETICU18_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU18_task-sedon1rest_eeg:25000

______________________ sub-NETICU19_task-sedon1rest_eeg ______________________

    - Session 20010124 1538.mff 
    - 2001-01-24 20:39:53.472000+00:00 
    - <RawMff | signal1.bin, 131 x 225105 (900.4 s), ~225.2 MB, data loaded> 
    - time(s): 900.42 
    - first_samp-last_samp: 0 - 225104
    - Duration = 0hr : 15min : 00sec
Name file for sub-NETICU19_task-sedon1rest_eeg:plot
Name for file sub-NETICU19_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=250.0) for sub-NETICU19_task-sedon1rest_eeg:1250

______________________ sub-NETICU19_task-sedon1taken_eeg ______________________

    - Session 20010124 1517.mff 
    - 2001-01-24 20:26:25.796000+00:00 
    - <RawMff | signal1.bin, 133 x 169182 (676.7 s), ~171.8 MB, data loaded> 
    - time(s): 676.728 
    - first_samp-last_samp: 0 - 169181
    - Duration = 0hr : 11min : 16sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1017 / 250.0 = 4.068
2 - 87329 / 250.0 = 349.316
Name event1 for sub-NETICU19_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU19_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU19_task-sedofftaken_eeg ______________________

    - Session 20010124 1639.mff 
    - 2001-01-24 21:41:01.168000+00:00 
    - <RawMff | signal1.bin, 133 x 169107 (676.4 s), ~171.8 MB, data loaded> 
    - time(s): 676.428 
    - first_samp-last_samp: 0 - 169106
    - Duration = 0hr : 11min : 16sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 951 / 250.0 = 3.804
2 - 87243 / 250.0 = 348.972
Name event1 for sub-NETICU19_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU19_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU19_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU19_task-sedoffrest_eeg ______________________

    - Session 20010124 1558.mff 
    - 2001-01-24 20:59:08.420000+00:00 
    - <RawMff | signal1.bin, 131 x 154045 (616.2 s), ~154.1 MB, data loaded> 
    - time(s): 616.18 
    - first_samp-last_samp: 0 - 154044
    - Duration = 0hr : 10min : 16sec
Name file for sub-NETICU19_task-sedoffrest_eeg:plot
Name for file sub-NETICU19_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=250.0) for sub-NETICU19_task-sedoffrest_eeg:30000

______________________ sub-NETICU20_task-sedoffrest_eeg ______________________

    - Session 20010212 2348.mff 
    - 2001-02-13 04:49:23.114000+00:00 
    - <RawMff | signal1.bin, 131 x 603129 (603.1 s), ~603.0 MB, data loaded> 
    - time(s): 603.129 
    - first_samp-last_samp: 0 - 603128
    - Duration = 0hr : 10min : 03sec
Name file for sub-NETICU20_task-sedoffrest_eeg:plot
Name for file sub-NETICU20_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU20_task-sedoffrest_eeg:278000

______________________ sub-NETICU20_task-sedon1rest_eeg ______________________

    - Session 20010212 2319.mff 
    - 2001-02-13 04:20:24.872000+00:00 
    - <RawMff | signal1.bin, 131 x 1440450 (1440.4 s), ~1.41 GB, data loaded> 
    - time(s): 1440.45 
    - first_samp-last_samp: 0 - 1440449
    - Duration = 0hr : 24min : 00sec
Name file for sub-NETICU20_task-sedon1rest_eeg:plot
Name for file sub-NETICU20_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU20_task-sedon1rest_eeg:200000

______________________ sub-NETICU20_task-sedon1taken_eeg ______________________

    - Session 20010212 2258.mff 
    - 2001-02-13 04:06:39.545000+00:00 
    - <RawMff | signal1.bin, 133 x 170870 (683.5 s), ~173.6 MB, data loaded> 
    - time(s): 683.48 
    - first_samp-last_samp: 0 - 170869
    - Duration = 0hr : 11min : 23sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 1377 / 250.0 = 5.508
2 - 86870 / 250.0 = 347.48
Name event1 for sub-NETICU20_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU20_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU22_task-sedon1taken_eeg ______________________

    - Session 20010322 0427.mff 
    - 2001-03-22 09:37:18.005000+00:00 
    - <RawMff | signal1.bin, 133 x 999764 (999.8 s), ~1014.6 MB, data loaded> 
    - time(s): 999.764 
    - first_samp-last_samp: 0 - 999763
    - Duration = 0hr : 16min : 39sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 3747 / 1000.0 = 3.747
2 - 295382 / 1000.0 = 295.382
3 - 667432 / 1000.0 = 667.432
Name event1 for sub-NETICU22_task-sedon1taken_eeg:plot
Name event1 for sub-NETICU22_task-sedon1taken_eeg:no
Name event2 for sub-NETICU22_task-sedon1taken_eeg:sedon1_ST
Name event3 for sub-NETICU22_task-sedon1taken_eeg:sedon1_IT

______________________ sub-NETICU22_task-sedon1rest_eeg ______________________

    - Session 20010322 0459.mff 
    - 2001-03-22 10:08:23.525000+00:00 
    - <RawMff | signal1.bin, 132 x 631179 (631.2 s), ~635.8 MB, data loaded> 
    - time(s): 631.179 
    - first_samp-last_samp: 0 - 631178
    - Duration = 0hr : 10min : 31sec
    - Trigger <  ch. 09  >
1 - 3736 / 1000.0 = 3.736
Name event1 for sub-NETICU22_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU22_task-sedoffrest_eeg ______________________

    - Session 20010322 0525.mff 
    - 2001-03-22 10:27:26.826000+00:00 
    - <RawMff | signal1.bin, 132 x 1807186 (1807.2 s), ~1.78 GB, data loaded> 
    - time(s): 1807.186 
    - first_samp-last_samp: 0 - 1807185
    - Duration = 0hr : 30min : 07sec
    - Trigger <  ch. 09  >
1 - 3778 / 1000.0 = 3.778
Name event1 for sub-NETICU22_task-sedoffrest_eeg:plot
Name event1 for sub-NETICU22_task-sedoffrest_eeg:sedoff_RS_smin
> smin to replace event1 start time (type "zoom" to plot a closer look):1465000
['NETICU22sedoff_RS_______________sub-NETICU22_task-sedoffrest_eeg']

______________________ sub-NETICU22_task-sedofftaken_eeg ______________________

    - Session 20010322 0558.mff 
    - 2001-03-22 11:08:26.744000+00:00 
    - <RawMff | signal1.bin, 133 x 185931 (743.7 s), ~188.8 MB, data loaded> 
    - time(s): 743.724 
    - first_samp-last_samp: 0 - 185930
    - Duration = 0hr : 12min : 23sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 10  >
1 - 998 / 250.0 = 3.992
2 - 100846 / 250.0 = 403.384
Name event1 for sub-NETICU22_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU22_task-sedofftaken_eeg:sedoff_IT

______________________ sub-NETICU24_task-sedon1taken_eeg ______________________

    - Session 20230112 1715.mff 
    - 2023-01-12 22:29:34.426000+00:00 
    - <RawMff | signal1.bin, 135 x 642931 (642.9 s), ~662.4 MB, data loaded> 
    - time(s): 642.931 
    - first_samp-last_samp: 0 - 642930
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 7256 / 1000.0 = 7.256
2 - 309942 / 1000.0 = 309.942
3 - 320146 / 1000.0 = 320.146
4 - 632362 / 1000.0 = 632.362

Name event1 for sub-NETICU24_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU24_task-sedon1taken_eeg:no
Name event3 for sub-NETICU24_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU24_task-sedon1taken_eeg:no

______________________ sub-NETICU24_task-sedon1rest_eeg ______________________

    - Session 20230112 1741.mff 
    - 2023-01-12 22:47:26.506000+00:00 
    - <RawMff | signal1.bin, 132 x 1334853 (1334.9 s), ~1.31 GB, data loaded> 
    - time(s): 1334.853 
    - first_samp-last_samp: 0 - 1334852
    - Duration = 0hr : 22min : 14sec
    - Trigger <  ch. 09  >
1 - 5066 / 1000.0 = 5.066
Name event1 for sub-NETICU24_task-sedon1rest_eeg:plot
Name event1 for sub-NETICU24_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU24_task-sedoffrest_eeg ______________________

    - Session 20230112 1811.mff 
    - 2023-01-12 23:18:24.552000+00:00 
    - <RawMff | signal1.bin, 133 x 611229 (611.2 s), ~620.4 MB, data loaded> 
    - time(s): 611.229 
    - first_samp-last_samp: 0 - 611228
    - Duration = 0hr : 10min : 11sec
    - Trigger <  ch. 18  >
    - Trigger <  ch. 28  >
1 - 5027 / 1000.0 = 5.027
2 - 605790 / 1000.0 = 605.79
Name event1 for sub-NETICU24_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU24_task-sedoffrest_eeg:no

______________________ sub-NETICU24_task-sedofftaken_eeg ______________________

    - Session 20230112 1829.mff 
    - 2023-01-12 23:31:04.419000+00:00 
    - <RawMff | signal1.bin, 135 x 642428 (642.4 s), ~661.9 MB, data loaded> 
    - time(s): 642.428 
    - first_samp-last_samp: 0 - 642427
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 48  >
    - Trigger <  ch. 58  >
    - Trigger <  ch. 40  >
    - Trigger <  ch. 50  >
1 - 6960 / 1000.0 = 6.96
2 - 309470 / 1000.0 = 309.47
3 - 319668 / 1000.0 = 319.668
4 - 631882 / 1000.0 = 631.882
Name event1 for sub-NETICU24_task-sedofftaken_eeg:plot
Name event1 for sub-NETICU24_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU24_task-sedofftaken_eeg:no
Name event3 for sub-NETICU24_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU24_task-sedofftaken_eeg:no

______________________ sub-NETICU25_task-sedon1taken_eeg ______________________

    - Session 20230120 1549.mff 
    - 2023-01-20 20:56:27.012000+00:00 
    - <RawMff | signal1.bin, 135 x 160869 (643.5 s), ~165.9 MB, data loaded> 
    - time(s): 643.476 
    - first_samp-last_samp: 0 - 160868
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 1901 / 250.0 = 7.604
2 - 77623 / 250.0 = 310.492
3 - 80175 / 250.0 = 320.7
4 - 158232 / 250.0 = 632.928
Name event1 for sub-NETICU25_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU25_task-sedon1taken_eeg:no
Name event3 for sub-NETICU25_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU25_task-sedon1taken_eeg:no

______________________ sub-NETICU25_task-sedon1rest_eeg ______________________

    - Session 20230120 1608.mff 
    - 2023-01-20 21:14:07.617000+00:00 
    - <RawMff | signal1.bin, 131 x 610478 (610.5 s), ~610.3 MB, data loaded> 
    - time(s): 610.478 
    - first_samp-last_samp: 0 - 610477
    - Duration = 0hr : 10min : 10sec
Name file for sub-NETICU25_task-sedon1rest_eeg:plot
Name for file sub-NETICU25_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU25_task-sedon1rest_eeg:5000

______________________ sub-NETICU25_task-sedoffrest_eeg ______________________

    - Session 20230120 1625.mff 
    - 2023-01-20 21:29:46.560000+00:00 
    - <RawMff | signal1.bin, 131 x 1213578 (1213.6 s), ~1.18 GB, data loaded> 
    - time(s): 1213.578 
    - first_samp-last_samp: 0 - 1213577
    - Duration = 0hr : 20min : 13sec
Name file for sub-NETICU25_task-sedoffrest_eeg:plot
Name for file sub-NETICU25_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU25_task-sedoffrest_eeg:871000

______________________ sub-NETICU27_task-sedon1taken_eeg ______________________

    - Session 20230419 0650.mff 
    - 2023-04-19 12:05:03.697000+00:00 
    - <RawMff | signal1.bin, 135 x 642666 (642.7 s), ~662.1 MB, data loaded> 
    - time(s): 642.666 
    - first_samp-last_samp: 0 - 642665
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 6802 / 1000.0 = 6.802
2 - 309464 / 1000.0 = 309.464
3 - 319957 / 1000.0 = 319.957
4 - 632162 / 1000.0 = 632.162
Name event1 for sub-NETICU27_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU27_task-sedon1taken_eeg:no
Name event3 for sub-NETICU27_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU27_task-sedon1taken_eeg:no

______________________ sub-NETICU27_task-sedon1rest_eeg ______________________

    - Session 20230419 0716.mff 
    - 2023-04-19 12:17:41.207000+00:00 
    - <RawMff | signal1.bin, 134 x 1336156 (1336.2 s), ~1.33 GB, data loaded> 
    - time(s): 1336.156 
    - first_samp-last_samp: 0 - 1336155
    - Duration = 0hr : 22min : 16sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 99  >
    - Trigger <  ch. 19  >
1 - 5038 / 1000.0 = 5.038
2 - 730725 / 1000.0 = 730.725
3 - 1330740 / 1000.0 = 1330.74
Name event1 for sub-NETICU27_task-sedon1rest_eeg:plot
Name event1 for sub-NETICU27_task-sedon1rest_eeg:sedon1_RS
Name event2 for sub-NETICU27_task-sedon1rest_eeg:no
Name event3 for sub-NETICU27_task-sedon1rest_eeg:no

______________________ sub-NETICU27_task-sedofftaken_eeg ______________________

    - Session 20230419 0808.mff 
    - 2023-04-19 13:10:42.998000+00:00 
    - <RawMff | signal1.bin, 135 x 642455 (642.5 s), ~661.9 MB, data loaded> 
    - time(s): 642.455 
    - first_samp-last_samp: 0 - 642454
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 48  >
    - Trigger <  ch. 58  >
    - Trigger <  ch. 40  >
    - Trigger <  ch. 50  >
1 - 6923 / 1000.0 = 6.923
2 - 309582 / 1000.0 = 309.582
3 - 319776 / 1000.0 = 319.776
4 - 631969 / 1000.0 = 631.969
Name event1 for sub-NETICU27_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU27_task-sedofftaken_eeg:no
Name event3 for sub-NETICU27_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU27_task-sedofftaken_eeg:no

______________________ sub-NETICU27_task-sedoffrest_eeg ______________________

    - Session 20230419 0740.mff 
    - 2023-04-19 12:56:43.790000+00:00 
    - <RawMff | signal1.bin, 133 x 611556 (611.6 s), ~620.7 MB, data loaded> 
    - time(s): 611.556 
    - first_samp-last_samp: 0 - 611555
    - Duration = 0hr : 10min : 11sec
    - Trigger <  ch. 18  >
    - Trigger <  ch. 28  >
1 - 6084 / 1000.0 = 6.084
2 - 606125 / 1000.0 = 606.125
Name event1 for sub-NETICU27_task-sedoffrest_eeg:plot
Name event1 for sub-NETICU27_task-sedoffrest_eeg:sedoff_RS_smin
> smin to replace event1 start time (type "zoom" to plot a closer look):278000
Name event2 for sub-NETICU27_task-sedoffrest_eeg:no

______________________ sub-NETICU28_task-sedofftaken_eeg ______________________

    - Session 20230426 1436.mff 
    - 2023-04-26 19:37:58.907000+00:00 
    - <RawMff | signal1.bin, 135 x 642355 (642.4 s), ~661.8 MB, data loaded> 
    - time(s): 642.355 
    - first_samp-last_samp: 0 - 642354
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 48  >
    - Trigger <  ch. 58  >
    - Trigger <  ch. 40  >
    - Trigger <  ch. 50  >
1 - 6818 / 1000.0 = 6.818
2 - 309475 / 1000.0 = 309.475
3 - 319651 / 1000.0 = 319.651
4 - 631842 / 1000.0 = 631.842
Name event1 for sub-NETICU28_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU28_task-sedofftaken_eeg:no
Name event3 for sub-NETICU28_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU28_task-sedofftaken_eeg:no

______________________ sub-NETICU28_task-sedon1taken_eeg ______________________

    - Session 20230426 1336.mff 
    - 2023-04-26 18:37:50.704000+00:00 
    - <RawMff | signal1.bin, 135 x 642304 (642.3 s), ~661.7 MB, data loaded> 
    - time(s): 642.304 
    - first_samp-last_samp: 0 - 642303
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 6944 / 1000.0 = 6.944
2 - 309419 / 1000.0 = 309.419
3 - 319609 / 1000.0 = 319.609
4 - 631802 / 1000.0 = 631.802
Name event1 for sub-NETICU28_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU28_task-sedon1taken_eeg:no
Name event3 for sub-NETICU28_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU28_task-sedon1taken_eeg:no

______________________ sub-NETICU28_task-sedoffrest_eeg ______________________

    - Session 20230426 1422.mff 
    - 2023-04-26 19:25:24.451000+00:00 
    - <RawMff | signal1.bin, 133 x 611507 (611.5 s), ~620.7 MB, data loaded> 
    - time(s): 611.507 
    - first_samp-last_samp: 0 - 611506
    - Duration = 0hr : 10min : 11sec
    - Trigger <  ch. 18  >
    - Trigger <  ch. 28  >
1 - 6055 / 1000.0 = 6.055
2 - 606089 / 1000.0 = 606.089
Name event1 for sub-NETICU28_task-sedoffrest_eeg:plot
Name event1 for sub-NETICU28_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU28_task-sedoffrest_eeg:no

______________________ sub-NETICU28_task-sedon1rest_eeg ______________________

    - Session 20230426 1348.mff 
    - 2023-04-26 18:49:54.103000+00:00 
    - <RawMff | signal1.bin, 132 x 1270261 (1270.3 s), ~1.25 GB, data loaded> 
    - time(s): 1270.261 
    - first_samp-last_samp: 0 - 1270260
    - Duration = 0hr : 21min : 10sec
    - Trigger <  ch. 09  >
1 - 5080 / 1000.0 = 5.08
Name event1 for sub-NETICU28_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU29_task-sedon1rest_eeg ______________________

    - Session 20230428 1736.mff 
    - 2023-04-28 22:46:28.526000+00:00 
    - <RawMff | signal1.bin, 131 x 615706 (615.7 s), ~615.5 MB, data loaded> 
    - time(s): 615.706 
    - first_samp-last_samp: 0 - 615705
    - Duration = 0hr : 10min : 15sec
Name file for sub-NETICU29_task-sedon1rest_eeg:plot
Name for file sub-NETICU29_task-sedon1rest_eeg:sedon1_RS
> smin value (samplingrate=1000.0) for sub-NETICU29_task-sedon1rest_eeg:5000

______________________ sub-NETICU29_task-sedoffrest_eeg ______________________

    - Session 20230428 1812.mff 
    - 2023-04-28 23:14:38.744000+00:00 
    - <RawMff | signal1.bin, 131 x 629154 (629.2 s), ~629.0 MB, data loaded> 
    - time(s): 629.154 
    - first_samp-last_samp: 0 - 629153
    - Duration = 0hr : 10min : 29sec
Name file for sub-NETICU29_task-sedoffrest_eeg:plot
Name for file sub-NETICU29_task-sedoffrest_eeg:sedoff_RS
> smin value (samplingrate=1000.0) for sub-NETICU29_task-sedoffrest_eeg:5000

______________________ sub-NETICU29_task-sedon1taken_eeg ______________________

    - Session 20230428 1716.mff 
    - 2023-04-28 22:25:06.172000+00:00 
    - <RawMff | signal1.bin, 135 x 159326 (637.3 s), ~164.3 MB, data loaded> 
    - time(s): 637.304 
    - first_samp-last_samp: 0 - 159325
    - Duration = 0hr : 10min : 37sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 1733 / 250.0 = 6.932
2 - 77353 / 250.0 = 309.412
3 - 79901 / 250.0 = 319.604
4 - 157949 / 250.0 = 631.796
Name event1 for sub-NETICU29_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU29_task-sedon1taken_eeg:no
Name event3 for sub-NETICU29_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU29_task-sedon1taken_eeg:no

______________________ sub-NETICU29_task-sedofftaken_eeg ______________________

    - Session 20230428 1826.mff 
    - 2023-04-28 23:27:53.852000+00:00 
    - <RawMff | signal1.bin, 135 x 159001 (636.0 s), ~163.9 MB, data loaded> 
    - time(s): 636.004 
    - first_samp-last_samp: 0 - 159000
    - Duration = 0hr : 10min : 36sec
    - Trigger <  ch. 48  >
    - Trigger <  ch. 58  >
    - Trigger <  ch. 40  >
    - Trigger <  ch. 50  >
1 - 1745 / 250.0 = 6.98
2 - 77412 / 250.0 = 309.648
3 - 79954 / 250.0 = 319.816
4 - 158003 / 250.0 = 632.012

Name event1 for sub-NETICU29_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU29_task-sedofftaken_eeg:no
Name event3 for sub-NETICU29_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU29_task-sedofftaken_eeg:no

______________________ sub-NETICU30_task-sedofftaken_eeg ______________________

    - Session 20230526 1042.mff 
    - 2023-05-26 15:43:19.104000+00:00 
    - <RawMff | signal1.bin, 135 x 642307 (642.3 s), ~661.7 MB, data loaded> 
    - time(s): 642.307 
    - first_samp-last_samp: 0 - 642306
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 48  >
    - Trigger <  ch. 58  >
    - Trigger <  ch. 40  >
    - Trigger <  ch. 50  >
1 - 6797 / 1000.0 = 6.797
2 - 309453 / 1000.0 = 309.453
3 - 319611 / 1000.0 = 319.611
4 - 631805 / 1000.0 = 631.805
Name event1 for sub-NETICU30_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU30_task-sedofftaken_eeg:no
Name event3 for sub-NETICU30_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU30_task-sedofftaken_eeg:no

______________________ sub-NETICU30_task-sedon1rest_eeg ______________________

    - Session 20230526 1007.mff 
    - 2023-05-26 15:08:26.312000+00:00 
    - <RawMff | signal1.bin, 132 x 1364653 (1364.7 s), ~1.34 GB, data loaded> 
    - time(s): 1364.653 
    - first_samp-last_samp: 0 - 1364652
    - Duration = 0hr : 22min : 44sec
    - Trigger <  ch. 09  >
1 - 5068 / 1000.0 = 5.068
Name event1 for sub-NETICU30_task-sedon1rest_eeg:plot
Name event1 for sub-NETICU30_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU30_task-sedon1taken_eeg ______________________

    - Session 20230526 0943.mff 
    - 2023-05-26 14:55:57.934000+00:00 
    - <RawMff | signal1.bin, 135 x 642658 (642.7 s), ~662.1 MB, data loaded> 
    - time(s): 642.658 
    - first_samp-last_samp: 0 - 642657
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 12  >
    - Trigger <  ch. 22  >
    - Trigger <  ch. 10  >
    - Trigger <  ch. 20  >
1 - 6927 / 1000.0 = 6.927
2 - 309410 / 1000.0 = 309.41
3 - 319965 / 1000.0 = 319.965
4 - 632157 / 1000.0 = 632.157
Name event1 for sub-NETICU30_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU30_task-sedon1taken_eeg:no
Name event3 for sub-NETICU30_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU30_task-sedon1taken_eeg:no

______________________ sub-NETICU30_task-sedoffrest_eeg ______________________

    - Session 20230526 1031.mff 
    - 2023-05-26 15:32:15.257000+00:00 
    - <RawMff | signal1.bin, 133 x 611505 (611.5 s), ~620.7 MB, data loaded> 
    - time(s): 611.505 
    - first_samp-last_samp: 0 - 611504
    - Duration = 0hr : 10min : 11sec
    - Trigger <  ch. 18  >
    - Trigger <  ch. 28  >
1 - 6066 / 1000.0 = 6.066
2 - 606083 / 1000.0 = 606.083
Name event1 for sub-NETICU30_task-sedoffrest_eeg:plot
Name event1 for sub-NETICU30_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU30_task-sedoffrest_eeg:no

______________________ sub-NETICU31_task-sedon1taken_eeg ______________________

    - Session 20240207 1713.mff 
    - 2024-02-07 22:21:05.907000+00:00 
    - <RawMff | signal1.bin, 135 x 643106 (643.1 s), ~662.6 MB, data loaded> 
    - time(s): 643.106 
    - first_samp-last_samp: 0 - 643105
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7610 / 1000.0 = 7.61
2 - 310094 / 1000.0 = 310.094
3 - 320244 / 1000.0 = 320.244
4 - 632508 / 1000.0 = 632.508
Name event1 for sub-NETICU31_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU31_task-sedon1taken_eeg:no
Name event3 for sub-NETICU31_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU31_task-sedon1taken_eeg:no
    
______________________ sub-NETICU31_task-sedon1rest_eeg ______________________

    - Session 20240207 1743.mff 
    - 2024-02-07 22:44:10.311000+00:00 
    - <RawMff | signal1.bin, 133 x 30751 (30.8 s), ~31.4 MB, data loaded> 
    - time(s): 30.751 
    - first_samp-last_samp: 0 - 30750
    - Duration = 0hr : 00min : 30sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5199 / 1000.0 = 5.199
2 - 25258 / 1000.0 = 25.258
Name event1 for sub-NETICU31_task-sedon1rest_eeg:no
Name event2 for sub-NETICU31_task-sedon1rest_eeg:no

______________________ sub-NETICU31_task-sedofftaken_eeg ______________________

    - Session 20240207 1807.mff 
    - 2024-02-07 23:09:41.046000+00:00 
    - <RawMff | signal1.bin, 135 x 643406 (643.4 s), ~662.9 MB, data loaded> 
    - time(s): 643.406 
    - first_samp-last_samp: 0 - 643405
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7962 / 1000.0 = 7.962
2 - 310459 / 1000.0 = 310.459
3 - 320613 / 1000.0 = 320.613
4 - 632810 / 1000.0 = 632.81
Name event1 for sub-NETICU31_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU31_task-sedofftaken_eeg:no
Name event3 for sub-NETICU31_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU31_task-sedofftaken_eeg:no
    
______________________ sub-NETICU31_task-sedon2rest_eeg ______________________

    - Session 20240207 1821.mff 
    - 2024-02-07 23:22:18.055000+00:00 
    - <RawMff | signal1.bin, 131 x 1210161 (1210.2 s), ~1.18 GB, data loaded> 
    - time(s): 1210.161 
    - first_samp-last_samp: 0 - 1210160
    - Duration = 0hr : 20min : 10sec
Name file for sub-NETICU31_task-sedon2rest_eeg:sedon2_RS
> smin value (samplingrate=1000.0) for sub-NETICU31_task-sedon2rest_eeg:870000

______________________ sub-NETICU33_task-sedoffrest_eeg ______________________

    - Session 20240224 1128.mff 
    - 2024-02-24 16:29:46.064000+00:00 
    - <RawMff | signal1.bin, 133 x 30597 (30.6 s), ~31.2 MB, data loaded> 
    - time(s): 30.597 
    - first_samp-last_samp: 0 - 30596
    - Duration = 0hr : 00min : 30sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5034 / 1000.0 = 5.034
2 - 25092 / 1000.0 = 25.092
Name event1 for sub-NETICU33_task-sedoffrest_eeg:no
Name event2 for sub-NETICU33_task-sedoffrest_eeg:no

______________________ sub-NETICU33_task-sedon1taken_eeg ______________________

    - Session 20240224 1048.mff 
    - 2024-02-24 15:53:45.296000+00:00 
    - <RawMff | signal1.bin, 135 x 160951 (643.8 s), ~166.0 MB, data loaded> 
    - time(s): 643.804 
    - first_samp-last_samp: 0 - 160950
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 2037 / 250.0 = 8.148
2 - 77707 / 250.0 = 310.828
3 - 80245 / 250.0 = 320.98
4 - 158294 / 250.0 = 633.176
Name event1 for sub-NETICU33_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU33_task-sedon1taken_eeg:no
Name event3 for sub-NETICU33_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU33_task-sedon1taken_eeg:no

______________________ sub-NETICU33_task-sedon1rest_eeg ______________________

    - Session 20240224 1106.mff 
    - 2024-02-24 16:07:53.599000+00:00 
    - <RawMff | signal1.bin, 131 x 1201214 (1201.2 s), ~1.17 GB, data loaded> 
    - time(s): 1201.214 
    - first_samp-last_samp: 0 - 1201213
    - Duration = 0hr : 20min : 01sec
Name file for sub-NETICU33_task-sedon1rest_eeg:sedon1_RS1st
> smin value (samplingrate=1000.0) for sub-NETICU33_task-sedon1rest_eeg:10000
> smin for RS2nd in sub-NETICU33_task-sedon1rest_eeg:875000

______________________ sub-NETICU38_task-sedon1rest_eeg ______________________

    - Session 20240308 1020.mff 
    - 2024-03-08 15:21:36.061000+00:00 
    - <RawMff | signal1.bin, 132 x 3068129 (3068.1 s), ~3.02 GB, data loaded> 
    - time(s): 3068.129 
    - first_samp-last_samp: 0 - 3068128
    - Duration = 0hr : 51min : 08sec
    - Trigger <  ch. 09  >
1 - 5046 / 1000.0 = 5.046
Name event1 for sub-NETICU38_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU38_task-sedon1taken_eeg ______________________

    - Session 20240308 1008.mff 
    - 2024-03-08 15:09:23.206000+00:00 
    - <RawMff | signal1.bin, 135 x 643905 (643.9 s), ~663.4 MB, data loaded> 
    - time(s): 643.905 
    - first_samp-last_samp: 0 - 643904
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8178 / 1000.0 = 8.178
2 - 310945 / 1000.0 = 310.945
3 - 321101 / 1000.0 = 321.101
4 - 633333 / 1000.0 = 633.333
Name event1 for sub-NETICU38_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU38_task-sedon1taken_eeg:no
Name event3 for sub-NETICU38_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU38_task-sedon1taken_eeg:no

______________________ sub-NETICU40_task-sedoffrest_eeg ______________________

    - Session 20240403 0820.mff 
    - 2024-04-03 13:21:02.107000+00:00 
    - <RawMff | signal1.bin, 133 x 610509 (610.5 s), ~619.7 MB, data loaded> 
    - time(s): 610.509 
    - first_samp-last_samp: 0 - 610508
    - Duration = 0hr : 10min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5030 / 1000.0 = 5.03
2 - 605084 / 1000.0 = 605.084

Name event1 for sub-NETICU40_task-sedoffrest_eeg:sedoff_RS
Name event2 for sub-NETICU40_task-sedoffrest_eeg:no

______________________ sub-NETICU40_task-sedon1taken_eeg ______________________

    - Session 20240403 0733.mff 
    - 2024-04-03 12:33:33.810000+00:00 
    - <RawMff | signal1.bin, 135 x 643706 (643.7 s), ~663.2 MB, data loaded> 
    - time(s): 643.706 
    - first_samp-last_samp: 0 - 643705
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8121 / 1000.0 = 8.121
2 - 310800 / 1000.0 = 310.8
3 - 320955 / 1000.0 = 320.955
4 - 633149 / 1000.0 = 633.149
Name event1 for sub-NETICU40_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU40_task-sedon1taken_eeg:no
Name event3 for sub-NETICU40_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU40_task-sedon1taken_eeg:no

______________________ sub-NETICU40_task-sedon1rest_eeg ______________________

    - Session 20240403 0744.mff 
    - 2024-04-03 12:45:02.412000+00:00 
    - <RawMff | signal1.bin, 132 x 1358802 (1358.8 s), ~1.34 GB, data loaded> 
    - time(s): 1358.802 
    - first_samp-last_samp: 0 - 1358801
    - Duration = 0hr : 22min : 38sec
    - Trigger <  ch. 09  >
1 - 5057 / 1000.0 = 5.057
Name event1 for sub-NETICU40_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU40_task-sedofftaken_eeg ______________________

    - Session 20240403 0832.mff 
    - 2024-04-03 13:32:54.655000+00:00 
    - <RawMff | signal1.bin, 135 x 643106 (643.1 s), ~662.6 MB, data loaded> 
    - time(s): 643.106 
    - first_samp-last_samp: 0 - 643105
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7529 / 1000.0 = 7.529
2 - 310186 / 1000.0 = 310.186
3 - 320339 / 1000.0 = 320.339
4 - 632531 / 1000.0 = 632.531
Name event1 for sub-NETICU40_task-sedofftaken_eeg:sedoff_ST
Name event2 for sub-NETICU40_task-sedofftaken_eeg:no
Name event3 for sub-NETICU40_task-sedofftaken_eeg:sedoff_IT
Name event4 for sub-NETICU40_task-sedofftaken_eeg:no

______________________ sub-NETICU41_task-sedon1taken_eeg ______________________

    - Session 20240402 1435.mff 
    - 2024-04-02 19:51:26.036000+00:00 
    - <RawMff | signal1.bin, 135 x 643754 (643.8 s), ~663.2 MB, data loaded> 
    - time(s): 643.754 
    - first_samp-last_samp: 0 - 643753
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7990 / 1000.0 = 7.99
2 - 310658 / 1000.0 = 310.658
3 - 320877 / 1000.0 = 320.877
4 - 633160 / 1000.0 = 633.16
Name event1 for sub-NETICU41_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU41_task-sedon1taken_eeg:no
Name event3 for sub-NETICU41_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU41_task-sedon1taken_eeg:no

______________________ sub-NETICU41_task-sedon2taken_eeg ______________________

    - Session 20240402 1628.mff 
    - 2024-04-02 21:28:38.307000+00:00 
    - <RawMff | signal1.bin, 135 x 642854 (642.9 s), ~662.3 MB, data loaded> 
    - time(s): 642.854 
    - first_samp-last_samp: 0 - 642853
    - Duration = 0hr : 10min : 42sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 7319 / 1000.0 = 7.319
2 - 309977 / 1000.0 = 309.977
3 - 320062 / 1000.0 = 320.062
4 - 632257 / 1000.0 = 632.257
Name event1 for sub-NETICU41_task-sedon2taken_eeg:sedon2_ST
Name event2 for sub-NETICU41_task-sedon2taken_eeg:no
Name event3 for sub-NETICU41_task-sedon2taken_eeg:sedon2_IT
Name event4 for sub-NETICU41_task-sedon2taken_eeg:no

______________________ sub-NETICU41_task-sedon2rest_eeg ______________________

    - Session 20240402 1601.mff 
    - 2024-04-02 21:06:41.133000+00:00 
    - <RawMff | signal1.bin, 133 x 1210562 (1210.6 s), ~1.20 GB, data loaded> 
    - time(s): 1210.562 
    - first_samp-last_samp: 0 - 1210561
    - Duration = 0hr : 20min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5073 / 1000.0 = 5.073
2 - 1205134 / 1000.0 = 1205.134
Name event1 for sub-NETICU41_task-sedon2rest_eeg:sedon2_RS_smin
> smin to replace event1 start time (type "zoom" to plot a closer look):600000
Name event2 for sub-NETICU41_task-sedon2rest_eeg:no

______________________ sub-NETICU41_task-sedon1rest_eeg ______________________

    - Session 20240402 1502.mff 
    - 2024-04-02 20:03:00.507000+00:00 
    - <RawMff | signal1.bin, 132 x 1388249 (1388.2 s), ~1.37 GB, data loaded> 
    - time(s): 1388.249 
    - first_samp-last_samp: 0 - 1388248
    - Duration = 0hr : 23min : 08sec
    - Trigger <  ch. 09  >
1 - 5039 / 1000.0 = 5.039
Name event1 for sub-NETICU41_task-sedon1rest_eeg:sedon1_RS

______________________ sub-NETICU41_task-sedofftaken_eeg ______________________

    - Session 20240402 1549.mff 
    - 2024-04-02 20:50:00.707000+00:00 
    - <RawMff | signal1.bin, 138 x 643055 (643.1 s), ~677.2 MB, data loaded> 
    - time(s): 643.055 
    - first_samp-last_samp: 0 - 643054
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. cal*>
    - Trigger <  ch. cal=>
    - Trigger <  ch. cal->
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 2654 / 1000.0 = 2.654
2 - 2909 / 1000.0 = 2.909
3 - 4053 / 1000.0 = 4.053
4 - 7484 / 1000.0 = 7.484
5 - 310141 / 1000.0 = 310.141
6 - 320296 / 1000.0 = 320.296
7 - 632489 / 1000.0 = 632.489
Name event1 for sub-NETICU41_task-sedofftaken_eeg:no
Name event2 for sub-NETICU41_task-sedofftaken_eeg:no
Name event3 for sub-NETICU41_task-sedofftaken_eeg:no
Name event4 for sub-NETICU41_task-sedofftaken_eeg:sedoff_ST
Name event5 for sub-NETICU41_task-sedofftaken_eeg:no
Name event6 for sub-NETICU41_task-sedofftaken_eeg:sedoff_IT
Name event7 for sub-NETICU41_task-sedofftaken_eeg:no

______________________ sub-NETICU41_task-sedoffrest_eeg ______________________

    - Session 20240402 1538.mff 
    - 2024-04-02 20:39:09.659000+00:00 
    - <RawMff | signal1.bin, 133 x 610558 (610.6 s), ~619.7 MB, data loaded> 
    - time(s): 610.558 
    - first_samp-last_samp: 0 - 610557
    - Duration = 0hr : 10min : 10sec
    - Trigger <  ch. 09  >
    - Trigger <  ch. 19  >
1 - 5064 / 1000.0 = 5.064
2 - 605118 / 1000.0 = 605.118
Name event1 for sub-NETICU41_task-sedoffrest_eeg:sedoff_RS_smin
> smin to replace event1 start time (type "zoom" to plot a closer look):278000
Name event2 for sub-NETICU41_task-sedoffrest_eeg:no

______________________ sub-NETICU46_task-sedon1taken_eeg ______________________

    - Session 20240422 1532.mff 
    - 2024-04-22 20:54:05.369000+00:00 
    - <RawMff | signal1.bin, 135 x 643756 (643.8 s), ~663.2 MB, data loaded> 
    - time(s): 643.756 
    - first_samp-last_samp: 0 - 643755
    - Duration = 0hr : 10min : 43sec
    - Trigger <  ch. 22  >
    - Trigger <  ch. 32  >
    - Trigger <  ch. 20  >
    - Trigger <  ch. 30  >
1 - 8322 / 1000.0 = 8.322
2 - 310804 / 1000.0 = 310.804
3 - 320958 / 1000.0 = 320.958
4 - 633156 / 1000.0 = 633.156
Name event1 for sub-NETICU46_task-sedon1taken_eeg:sedon1_ST
Name event2 for sub-NETICU46_task-sedon1taken_eeg:no
Name event3 for sub-NETICU46_task-sedon1taken_eeg:sedon1_IT
Name event4 for sub-NETICU46_task-sedon1taken_eeg:no

______________________ sub-NETICU46_task-sedon1rest_eeg ______________________

    - Session 20240422 1605.mff 
    - 2024-04-22 21:05:58.257000+00:00 
    - <RawMff | signal1.bin, 132 x 1210613 (1210.6 s), ~1.19 GB, data loaded> 
    - time(s): 1210.613 
    - first_samp-last_samp: 0 - 1210612
    - Duration = 0hr : 20min : 10sec
    - Trigger <  ch. 09  >
1 - 5032 / 1000.0 = 5.032
Name event1 for sub-NETICU46_task-sedon1rest_eeg:sedon1_RS1st
> smin for RS2nd in sub-NETICU46_task-sedon1rest_eeg:883612




####################################################################
### 'no' to skip this event; 'exit' to stop code; 'plot' to plot ###
### ; name+'_smin' to set dif smin time; 'zoom' for second plot  ###
####################################################################







________________________________________________________________________________
    Saved Variables:
  ['HC01_ST_______________sub-HC01_task-ST_eeg',
 'HC01_IT_______________sub-HC01_task-IT_eeg',
 'HC02_ST_______________sub-HC02_task-Taken_eeg',
 'HC02_IT_______________sub-HC02_task-Taken_eeg',
 'HC03_IT_______________sub-HC03_task-Taken_eeg',
 'HC03_ST_______________sub-HC03_task-Taken_eeg',
 'HC04_ST_______________sub-HC04_task-Taken_eeg',
 'HC04_IT_______________sub-HC04_task-Taken_eeg',
 'HC05_ST_______________sub-HC05_task-Taken_eeg',
 'HC05_IT1st_______________sub-HC05_task-Taken_eeg',
 'HC05_IT2nd_______________sub-HC05_task-Taken_eeg',
 'HC06_ST_______________sub-HC06_task-Taken_eeg',
 'HC06_IT_______________sub-HC06_task-Taken_eeg',
 'HC07_ST_______________sub-HC07_task-Taken_eeg',
 'HC07_IT_______________sub-HC07_task-Taken_eeg',
 'HC08_ST_______________sub-HC08_task-Taken_eeg',
 'HC08_IT_______________sub-HC08_task-Taken_eeg',
 'HC09_ST_______________sub-HC09_task-Taken_eeg',
 'HC09_IT_______________sub-HC09_task-Taken_eeg',
 'HC10_ST_______________sub-HC10_task-Taken_eeg',
 'HC10_IT_______________sub-HC10_task-Taken_eeg',
 'HC11_ST_______________sub-HC11_task-Taken_eeg',
 'HC11_IT_______________sub-HC11_task-Taken_eeg',
 'HC12_ST_______________sub-HC12_task-Taken_eeg',
 'HC12_IT_______________sub-HC12_task-Taken_eeg',
 'HC13_ST_______________sub-HC13_task-Taken_eeg',
 'HC13_IT_______________sub-HC13_task-Taken_eeg',
 'HC14_ST_______________sub-HC14_task-Taken_eeg',
 'HC14_IT_______________sub-HC14_task-Taken_eeg',
 'HC15_ST_______________sub-HC15_task-Taken_eeg',
 'HC15_IT_______________sub-HC15_task-Taken_eeg',
 'HC16_ST_______________sub-HC16_task-Taken_eeg',
 'HC16_IT_______________sub-HC16_task-Taken_eeg',
 'HC17_ST_______________sub-HC17_task-Taken_eeg',
 'HC17_IT_______________sub-HC17_task-Taken_eeg',
 'HC18_ST_______________sub-HC18_task-Taken_eeg',
 'HC18_IT_______________sub-HC18_task-Taken_eeg']
#--------------------------------------------------------
['CCTC07_RS_______________sub-CCTC07_task-rest_eeg',
 'CCTC07_ST_______________sub-CCTC07_task-taken_eeg',
 'CCTC07_IT_______________sub-CCTC07_task-taken_eeg',
 'CCTC07NETICUsedon1_RS_______________sub-CCTC07NETICU_task-sedon1rest_eeg',
 'CCTC07NETICUsedoff_RS_______________sub-CCTC07NETICU_task-sedoffrest_eeg',
 'CCTC07NETICUsedoff_ST_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedoff_IT_______________sub-CCTC07NETICU_task-sedofftaken_eeg',
 'CCTC07NETICUsedon1_ST_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'CCTC07NETICUsedon1_IT_______________sub-CCTC07NETICU_task-sedon1taken_eeg',
 'MSICU32_RS_______________sub-MSICU32_task-rest_eeg',
 'MSICU32_ST_______________sub-MSICU32_task-taken_eeg',
 'MSICU32_IT_______________sub-MSICU32_task-taken_eeg',
 'MSICU38_ST_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_IT_______________sub-MSICU38_task-taken_eeg',
 'MSICU38_RS_______________sub-MSICU38_task-rest_eeg',
 'MSICU42_RS_______________sub-MSICU42_task-rest_eeg',
 'MSICU42_ST_______________sub-MSICU42_task-taken_eeg',
 'MSICU42_IT_______________sub-MSICU42_task-taken_eeg',
 'MSICU43_RS_______________sub-MSICU43_task-rest_eeg',
 'MSICU43_ST_______________sub-MSICU43_task-taken_eeg',
 'MSICU43_IT_______________sub-MSICU43_task-taken_eeg',
 'MSICU47_RS_______________sub-MSICU47_task-rest_eeg',
 'MSICU47_ST_______________sub-MSICU47_task-taken_eeg',
 'MSICU47_IT_______________sub-MSICU47_task-taken_eeg',
 'MSICU47np_______________sub-MSICU47_task-taken_eeg', #<--------------MISTAKE (delete)
#--------------------------------------------------------









________________________________________________________________________________
________________________________________________________________________________
    Saved Variables:
'NETICU01sedon1_ST_______________sub-NETICU01_task-sedon1taken_eeg',
'NETICU01sedon1_IT_______________sub-NETICU01_task-sedon1taken_eeg',
'NETICU01sedoff_ST_______________sub-NETICU01_task-sedofftaken_eeg',
'NETICU01sedoff_IT_______________sub-NETICU01_task-sedofftaken_eeg',
'NETICU01sedon1_RS_______________sub-NETICU01_task-sedon1rest_eeg',
'NETICU01sedoff_RS_______________sub-NETICU01_task-sedoffrest_eeg',
'NETICU02sedoff_RS_______________sub-NETICU02_task-sedoffrest_eeg',
'NETICU02sedon1_ST_______________sub-NETICU02_task-sedon1taken_eeg',
'NETICU02sedon1_IT_______________sub-NETICU02_task-sedon1taken_eeg',
'NETICU02sedon1_RS_______________sub-NETICU02_task-sedon1rest_eeg',
'NETICU02sedoff_IT_______________sub-NETICU02_task-sedofftaken_eeg',
'NETICU02sedoff_ST_______________sub-NETICU02_task-sedofftaken_eeg',
'NETICU03sedon1_RS_______________sub-NETICU03_task-sedon1rest_eeg',
'NETICU03sedoff_ST_______________sub-NETICU03_task-sedofftaken_eeg',
'NETICU03sedoff_IT_______________sub-NETICU03_task-sedofftaken_eeg',
'NETICU03sedon1_ST_______________sub-NETICU03_task-sedon1taken_eeg',
'NETICU03sedon1_IT_______________sub-NETICU03_task-sedon1taken_eeg',
'NETICU03sedoff_RS_______________sub-NETICU03_task-sedoffrest_eeg',
'NETICU04sedon1_ST_______________sub-NETICU04_task-sedon1taken_eeg',
'NETICU04sedon1_IT_______________sub-NETICU04_task-sedon1taken_eeg',
'NETICU04sedoff_RS_______________sub-NETICU04_task-sedoffrest_eeg',
'NETICU04sedoff_ST_______________sub-NETICU04_task-sedofftaken_eeg',
'NETICU04sedoff_IT_______________sub-NETICU04_task-sedofftaken_eeg',
'NETICU04sedon1_RS_______________sub-NETICU04_task-sedon1rest_eeg',
'NETICU05sedoff_RS_______________sub-NETICU05_task-sedoffrest_eeg',
'NETICU05sedon1_ST_______________sub-NETICU05_task-sedon1taken_eeg',
'NETICU05sedon1_IT_______________sub-NETICU05_task-sedon1taken_eeg',
'NETICU05sedon1_RS_______________sub-NETICU05_task-sedon1rest_eeg',
'NETICU05sedoff_ST_______________sub-NETICU05_task-sedofftaken_eeg',
'NETICU05sedoff_IT_______________sub-NETICU05_task-sedofftaken_eeg',
'NETICU06sedon1_ST_______________sub-NETICU06_task-sedon1taken_eeg',
'NETICU06sedon1_IT_______________sub-NETICU06_task-sedon1taken_eeg',
'NETICU06sedoff_RS_______________sub-NETICU06_task-sedoffrest_eeg',
'NETICU06sedoff_ST_______________sub-NETICU06_task-sedofftaken_eeg',
'NETICU06sedoff_IT_______________sub-NETICU06_task-sedofftaken_eeg',
'NETICU06sedon1_RS_______________sub-NETICU06_task-sedon1rest_eeg',
'NETICU08sedon2_RS_______________sub-NETICU08_task-sedon2rest_eeg',
'NETICU08sedoff_IT_______________sub-NETICU08_task-sedofftaken_eeg',
'NETICU08sedoff_ST_______________sub-NETICU08_task-sedofftaken_eeg',
'NETICU08sedon2_ST_______________sub-NETICU08_task-sedon2taken_eeg',
'NETICU08sedon2_IT_______________sub-NETICU08_task-sedon2taken_eeg',
 'NETICU11sedon1_ST_______________sub-NETICU11_task-sedon1taken_eeg',
 'NETICU11sedon1_IT_______________sub-NETICU11_task-sedon1taken_eeg',
 'NETICU11sedoff_RS_______________sub-NETICU11_task-sedoffrest_eeg',
 'NETICU11sedon1_RS_______________sub-NETICU11_task-sedon1rest_eeg',
 'NETICU11sedoff_ST_______________sub-NETICU11_task-sedofftaken_eeg',
 'NETICU11sedoff_IT_______________sub-NETICU11_task-sedofftaken_eeg',
 'NETICU13sedon1_ST_______________sub-NETICU13_task-sedon1taken_eeg',
 'NETICU13sedon1_IT_______________sub-NETICU13_task-sedon1taken_eeg',
 'NETICU13sedon1_RS_______________sub-NETICU13_task-sedon1rest_eeg',
 'NETICU13sedoff_ST_______________sub-NETICU13_task-sedofftaken_eeg',
 'NETICU13sedoff_IT_______________sub-NETICU13_task-sedofftaken_eeg',
 'NETICU13sedoff_RS_______________sub-NETICU13_task-sedoffrest_eeg',
 'NETICU15sedon1_RS_______________sub-NETICU15_task-sedon1rest_eeg',
 'NETICU15sedon1_ST_______________sub-NETICU15_task-sedon1taken_eeg',
 'NETICU15sedon1_IT_______________sub-NETICU15_task-sedon1taken_eeg',
 'NETICU15sedoff_ST_______________sub-NETICU15_task-sedofftaken_eeg',
 'NETICU15sedoff_IT_______________sub-NETICU15_task-sedofftaken_eeg',
 'NETICU15sedoff_RS_______________sub-NETICU15_task-sedoffrest_eeg',
 'NETICU17sedoff_IT_______________sub-NETICU17_task-sedofftaken_eeg',
 'NETICU17sedoff_ST_______________sub-NETICU17_task-sedofftaken_eeg',
 'NETICU17sedon1_RS_______________sub-NETICU17_task-sedon1rest_eeg',
 'NETICU17sedoff_RS_______________sub-NETICU17_task-sedoffrest_eeg',
 'NETICU17sedon1_ST_______________sub-NETICU17_task-sedon1taken_eeg',
 'NETICU17sedon1_IT_______________sub-NETICU17_task-sedon1taken_eeg',
 'NETICU18sedon1_ST_______________sub-NETICU18_task-sedon1taken_eeg',
 'NETICU18sedon1_IT_______________sub-NETICU18_task-sedon1taken_eeg',
 'NETICU18sedoff_RS_______________sub-NETICU18_task-sedoffrest_eeg',
 'NETICU18sedon1_RS_______________sub-NETICU18_task-sedon1rest_eeg',
 'NETICU19sedon1_RS_______________sub-NETICU19_task-sedon1rest_eeg',
 'NETICU19sedon1_ST_______________sub-NETICU19_task-sedon1taken_eeg',
 'NETICU19sedon1_IT_______________sub-NETICU19_task-sedon1taken_eeg',
 'NETICU19sedoff_ST_______________sub-NETICU19_task-sedofftaken_eeg',
 'NETICU19sedoff_IT_______________sub-NETICU19_task-sedofftaken_eeg',
 'NETICU19sedoff_RS_______________sub-NETICU19_task-sedoffrest_eeg',
 'NETICU20sedoff_RS_______________sub-NETICU20_task-sedoffrest_eeg',
 'NETICU20sedon1_RS_______________sub-NETICU20_task-sedon1rest_eeg',
 'NETICU20sedon1_ST_______________sub-NETICU20_task-sedon1taken_eeg',
 'NETICU20sedon1_IT_______________sub-NETICU20_task-sedon1taken_eeg,,
 'NETICU22sedon1_ST_______________sub-NETICU22_task-sedon1taken_eeg',
 'NETICU22sedon1_IT_______________sub-NETICU22_task-sedon1taken_eeg',
 'NETICU22sedon1_RS_______________sub-NETICU22_task-sedon1rest_eeg',
 'NETICU22sedoff_RS_______________sub-NETICU22_task-sedoffrest_eeg',
 'NETICU22sedoff_ST_______________sub-NETICU22_task-sedofftaken_eeg',
 'NETICU22sedoff_IT_______________sub-NETICU22_task-sedofftaken_eeg',
 'NETICU24sedon1_ST_______________sub-NETICU24_task-sedon1taken_eeg',
 'NETICU24sedon1_IT_______________sub-NETICU24_task-sedon1taken_eeg',
 'NETICU24sedon1_RS_______________sub-NETICU24_task-sedon1rest_eeg',
 'NETICU24sedoff_RS_______________sub-NETICU24_task-sedoffrest_eeg',
 'NETICU24sedoff_ST_______________sub-NETICU24_task-sedofftaken_eeg',
 'NETICU24sedoff_IT_______________sub-NETICU24_task-sedofftaken_eeg'
 'NETICU25sedon1_ST_______________sub-NETICU25_task-sedon1taken_eeg',
 'NETICU25sedon1_IT_______________sub-NETICU25_task-sedon1taken_eeg',
 'NETICU25sedon1_RS_______________sub-NETICU25_task-sedon1rest_eeg',
 'NETICU25sedoff_RS_______________sub-NETICU25_task-sedoffrest_eeg',
 'NETICU27sedon1_ST_______________sub-NETICU27_task-sedon1taken_eeg',
 'NETICU27sedon1_IT_______________sub-NETICU27_task-sedon1taken_eeg',
 'NETICU27sedon1_RS_______________sub-NETICU27_task-sedon1rest_eeg',
 'NETICU27sedoff_ST_______________sub-NETICU27_task-sedofftaken_eeg',
 'NETICU27sedoff_IT_______________sub-NETICU27_task-sedofftaken_eeg',
 'NETICU27sedoff_RS_______________sub-NETICU27_task-sedoffrest_eeg',
 'NETICU28sedoff_ST_______________sub-NETICU28_task-sedofftaken_eeg',
 'NETICU28sedoff_IT_______________sub-NETICU28_task-sedofftaken_eeg',
 'NETICU28sedon1_ST_______________sub-NETICU28_task-sedon1taken_eeg',
 'NETICU28sedon1_IT_______________sub-NETICU28_task-sedon1taken_eeg',
 'NETICU28sedoff_RS_______________sub-NETICU28_task-sedoffrest_eeg',
 'NETICU28sedon1_RS_______________sub-NETICU28_task-sedon1rest_eeg',
 'NETICU29sedon1_RS_______________sub-NETICU29_task-sedon1rest_eeg',
 'NETICU29sedoff_RS_______________sub-NETICU29_task-sedoffrest_eeg',
 'NETICU29sedon1_ST_______________sub-NETICU29_task-sedon1taken_eeg',
 'NETICU29sedon1_IT_______________sub-NETICU29_task-sedon1taken_eeg',
 'NETICU29sedoff_ST_______________sub-NETICU29_task-sedofftaken_eeg',
 'NETICU29sedoff_IT_______________sub-NETICU29_task-sedofftaken_eeg',
 'NETICU30sedoff_ST_______________sub-NETICU30_task-sedofftaken_eeg',
 'NETICU30sedoff_IT_______________sub-NETICU30_task-sedofftaken_eeg',
 'NETICU30sedon1_RS_______________sub-NETICU30_task-sedon1rest_eeg',
 'NETICU30sedon1_ST_______________sub-NETICU30_task-sedon1taken_eeg',
 'NETICU30sedon1_IT_______________sub-NETICU30_task-sedon1taken_eeg',
 'NETICU30sedoff_RS_______________sub-NETICU30_task-sedoffrest_eeg',
 'NETICU31sedoff_ST_______________sub-NETICU31_task-sedofftaken_eeg',
 'NETICU31sedoff_IT_______________sub-NETICU31_task-sedofftaken_eeg',
 'NETICU31sedon2_RS_______________sub-NETICU31_task-sedon2rest_eeg',
 'NETICU31sedon1_ST_______________sub-NETICU31_task-sedon1taken_eeg',
 'NETICU31sedon1_IT_______________sub-NETICU31_task-sedon1taken_eeg',
 'NETICU33sedon1_ST_______________sub-NETICU33_task-sedon1taken_eeg',
 'NETICU33sedon1_IT_______________sub-NETICU33_task-sedon1taken_eeg',
 'NETICU33sedon1_RS1st_______________sub-NETICU33_task-sedon1rest_eeg', #<---sedon1_RS*
 'NETICU33sedon1_RS2nd_______________sub-NETICU33_task-sedon1rest_eeg', #<---sedoff_RS*
 'NETICU38sedon1_RS_______________sub-NETICU38_task-sedon1rest_eeg',
 'NETICU38sedon1_ST_______________sub-NETICU38_task-sedon1taken_eeg',
 'NETICU38sedon1_IT_______________sub-NETICU38_task-sedon1taken_eeg',
 'NETICU40sedoff_RS_______________sub-NETICU40_task-sedoffrest_eeg',
 'NETICU40sedon1_ST_______________sub-NETICU40_task-sedon1taken_eeg',
 'NETICU40sedon1_IT_______________sub-NETICU40_task-sedon1taken_eeg',
 'NETICU40sedon1_RS_______________sub-NETICU40_task-sedon1rest_eeg',
 'NETICU40sedoff_ST_______________sub-NETICU40_task-sedofftaken_eeg',
 'NETICU40sedoff_IT_______________sub-NETICU40_task-sedofftaken_eeg',
 'NETICU41sedon1_ST_______________sub-NETICU41_task-sedon1taken_eeg',
 'NETICU41sedon1_IT_______________sub-NETICU41_task-sedon1taken_eeg',
 'NETICU41sedon2_ST_______________sub-NETICU41_task-sedon2taken_eeg',
 'NETICU41sedon2_IT_______________sub-NETICU41_task-sedon2taken_eeg',
 'NETICU41sedon2_RS_______________sub-NETICU41_task-sedon2rest_eeg',
 'NETICU41sedon1_RS_______________sub-NETICU41_task-sedon1rest_eeg',
 'NETICU41sedoff_ST_______________sub-NETICU41_task-sedofftaken_eeg',
 'NETICU41sedoff_IT_______________sub-NETICU41_task-sedofftaken_eeg',
 'NETICU41sedoff_RS_______________sub-NETICU41_task-sedoffrest_eeg',
 'NETICU46sedon1_ST_______________sub-NETICU46_task-sedon1taken_eeg',
 'NETICU46sedon1_IT_______________sub-NETICU46_task-sedon1taken_eeg',
 'NETICU46sedon1_RS1st_______________sub-NETICU46_task-sedon1rest_eeg',#<---sedon1_RS*
 'NETICU46sedon1_RS2nd_______________sub-NETICU46_task-sedon1rest_eeg']#<---sedoff_RS*
________________________________________________________________________________
________________________________________________________________________________




IndentationError: unindent does not match any outer indentation level (<string>, line 2867)